In [1]:
import torch
# GPUの名称、CUDA Compute Capabilityを取得
print(torch.cuda.get_device_name())

NVIDIA GeForce RTX 2080 Ti


## CSV読み込み

In [2]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules

from Baseline import Baseline

from FeatureEngineering import FeatureEngineering

from sqlalchemy import null, true

%matplotlib inline

gc.enable()

print("OK\n")

OK



In [3]:
"""
params_base = {
            'boosting_type': 'gbdt'
            ,'objective': 'binary'
            ,'metric': 'auc'
            ,'verbosity': -1
            ,'learning_rate': 0.05
            ,'n_estimators': 100000
            ,'random_state':123
            ,'bagging_freq': 1   
}
"""
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}


def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "num_leaves": trial.suggest_int("num_leaves", 8, 256)
            ,"min_child_samples": trial.suggest_int("min_child_samples", 5, 200)
            ,"min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True)
            ,"feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0)
            ,"bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0)
            ,"lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e+2, log=True)
            ,"lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e+2, log=True)
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
                    ,early_stopping_rounds=100
                    ,verbose=0
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

In [4]:
df_train_origin = pd.read_csv('./home-credit-default-risk/exports/df_train.csv')
df_test_origin =  pd.read_csv('./home-credit-default-risk/exports/df_test.csv')
df_train_origin.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
0,100002,1,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,1.0000,0.00000,0.00000,0.0,0.0000,0.0,0.0,0.0,0.0,19.0
1,100003,0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,0.9287,0.07140,0.00000,0.0,0.0000,0.0,0.0,0.0,0.0,28.0
2,100004,0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,0.7500,0.25000,0.00000,0.0,0.0000,0.0,0.0,0.0,0.0,4.0
3,100006,0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0.8570,0.09520,0.00000,0.0,0.0476,0.0,0.0,0.0,0.0,21.0
4,100007,0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,0.9395,0.04544,0.01515,0.0,0.0000,0.0,0.0,0.0,0.0,66.0


## 前処理

In [5]:
# infをnanへ置換
df_train_origin = df_train_origin.replace([np.inf, -np.inf], np.nan)
df_test_origin = df_test_origin.replace([np.inf, -np.inf], np.nan)
df_train_origin.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
0,100002,1,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,1.0000,0.00000,0.00000,0.0,0.0000,0.0,0.0,0.0,0.0,19.0
1,100003,0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,0.9287,0.07140,0.00000,0.0,0.0000,0.0,0.0,0.0,0.0,28.0
2,100004,0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,0.7500,0.25000,0.00000,0.0,0.0000,0.0,0.0,0.0,0.0,4.0
3,100006,0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0.8570,0.09520,0.00000,0.0,0.0476,0.0,0.0,0.0,0.0,21.0
4,100007,0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,0.9395,0.04544,0.01515,0.0,0.0000,0.0,0.0,0.0,0.0,66.0


In [6]:
#df_train_origin = df_train_origin.fillna(0)
#df_test_origin = df_test_origin.fillna(0)
#df_train_origin.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
0,100002,1,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,1.0000,0.00000,0.00000,0.0,0.0000,0.0,0.0,0.0,0.0,19.0
1,100003,0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,0.9287,0.07140,0.00000,0.0,0.0000,0.0,0.0,0.0,0.0,28.0
2,100004,0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,0.7500,0.25000,0.00000,0.0,0.0000,0.0,0.0,0.0,0.0,4.0
3,100006,0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0.8570,0.09520,0.00000,0.0,0.0476,0.0,0.0,0.0,0.0,21.0
4,100007,0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,0.9395,0.04544,0.01515,0.0,0.0000,0.0,0.0,0.0,0.0,66.0


In [7]:
for col in df_train_origin.columns:
    df_train_origin[col] = df_train_origin[col].replace('\s', '_', regex=True)
    df_train_origin[col] = df_train_origin[col].replace(':', '_', regex=True)
    df_train_origin[col] = df_train_origin[col].replace('/', '_', regex=True)
    df_train_origin[col] = df_train_origin[col].replace(r'\(', r'_', regex=True)
    df_train_origin[col] = df_train_origin[col].replace(r'\)', r'_', regex=True)
    df_train_origin[col] = df_train_origin[col].replace(r'\+', r'_', regex=True)

df_train_origin.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
0,100002,1,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,1.0000,0.00000,0.00000,0.0,0.0000,0.0,0.0,0.0,0.0,19.0
1,100003,0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,0.9287,0.07140,0.00000,0.0,0.0000,0.0,0.0,0.0,0.0,28.0
2,100004,0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,0.7500,0.25000,0.00000,0.0,0.0000,0.0,0.0,0.0,0.0,4.0
3,100006,0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0.8570,0.09520,0.00000,0.0,0.0476,0.0,0.0,0.0,0.0,21.0
4,100007,0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,0.9395,0.04544,0.01515,0.0,0.0000,0.0,0.0,0.0,0.0,66.0


In [8]:
for col in df_test_origin.columns:
    df_test_origin[col] = df_test_origin[col].replace('\s', '_', regex=True)
    df_test_origin[col] = df_test_origin[col].replace(':', '_', regex=True)
    df_test_origin[col] = df_test_origin[col].replace('/', '_', regex=True)
    df_test_origin[col] = df_test_origin[col].replace(r'\(', r'_', regex=True)
    df_test_origin[col] = df_test_origin[col].replace(r'\)', r'_', regex=True)
    df_test_origin[col] = df_test_origin[col].replace(r'\+', r'_', regex=True)

df_test_origin.head()

,SK_ID_CURR,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
0,100001,0,0,0,0,135000.0,568800.0,20560.5,450000.0,0.01884,...,0.7780,0.2222,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,9.0
1,100005,1,0,0,0,99000.0,222768.0,17370.0,180000.0,0.03580,...,0.8184,0.0909,0.09090,0.0,0.0,0.0,0.0,0.0,0.0,11.0
2,100013,1,1,0,0,202500.0,663264.0,69777.0,630000.0,0.01910,...,0.8887,0.0833,0.02777,0.0,0.0,0.0,0.0,0.0,0.0,36.0
3,100028,0,0,0,2,315000.0,1575000.0,49018.5,1575000.0,0.02640,...,0.9355,0.0645,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,31.0
4,100038,1,1,1,1,180000.0,625500.0,32067.0,625500.0,0.01003,...,0.9230,0.0769,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,13.0


In [9]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')

In [10]:
encoding_columns = []

for col in df_test_origin.columns:
    print(col)
    encoding_columns.append(col)
    
encoding_columns.remove('SK_ID_CURR')
encoding_columns.remove('EXT_SOURCE_3')
encoding_columns.remove('EXT_SOURCE_2')
encoding_columns.remove('EXT_SOURCE_1')

SK_ID_CURR
CODE_GENDER
FLAG_OWN_CAR
FLAG_OWN_REALTY
CNT_CHILDREN
AMT_INCOME_TOTAL
AMT_CREDIT
AMT_ANNUITY
AMT_GOODS_PRICE
REGION_POPULATION_RELATIVE
DAYS_BIRTH
DAYS_EMPLOYED
DAYS_REGISTRATION
DAYS_ID_PUBLISH
OWN_CAR_AGE
FLAG_MOBIL
FLAG_EMP_PHONE
FLAG_WORK_PHONE
FLAG_CONT_MOBILE
FLAG_PHONE
FLAG_EMAIL
CNT_FAM_MEMBERS
REGION_RATING_CLIENT
REGION_RATING_CLIENT_W_CITY
HOUR_APPR_PROCESS_START
REG_REGION_NOT_LIVE_REGION
REG_REGION_NOT_WORK_REGION
LIVE_REGION_NOT_WORK_REGION
REG_CITY_NOT_LIVE_CITY
REG_CITY_NOT_WORK_CITY
LIVE_CITY_NOT_WORK_CITY
EXT_SOURCE_1
EXT_SOURCE_2
EXT_SOURCE_3
APARTMENTS_AVG
BASEMENTAREA_AVG
YEARS_BEGINEXPLUATATION_AVG
YEARS_BUILD_AVG
COMMONAREA_AVG
ELEVATORS_AVG
ENTRANCES_AVG
FLOORSMAX_AVG
FLOORSMIN_AVG
LANDAREA_AVG
LIVINGAPARTMENTS_AVG
LIVINGAREA_AVG
NONLIVINGAPARTMENTS_AVG
NONLIVINGAREA_AVG
APARTMENTS_MODE
BASEMENTAREA_MODE
YEARS_BEGINEXPLUATATION_MODE
YEARS_BUILD_MODE
COMMONAREA_MODE
ELEVATORS_MODE
ENTRANCES_MODE
FLOORSMAX_MODE
FLOORSMIN_MODE
LANDAREA_MODE
LIVINGAPARTM

In [11]:
### Yao-Jhonson変換
#for column in encoding_columns:
#    modules.process_yj_convert(df_train_origin, df_test_origin, column, [column])
#
#encoding_columns.append('EXT_SOURCE_3')
#encoding_columns.append('EXT_SOURCE_2')
#encoding_columns.append('EXT_SOURCE_1')
#df_train_origin.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
0,100002,1,-1.388135,-0.717914,-0.664531,-0.653139,0.647368,-0.284044,0.028202,-0.315489,...,1.625994,-1.653296,-0.476503,-0.113778,-0.118523,-0.031869,-0.006247,-0.00255,-0.005981,-0.118159
1,100003,0,0.720391,-0.717914,1.504820,-0.653139,1.212877,1.495411,0.744282,1.406475,...,0.360092,0.055439,-0.476503,-0.113778,-0.118523,-0.031869,-0.006247,-0.00255,-0.005981,0.298617
2,100004,0,-1.388135,1.392925,-0.664531,-0.653139,-1.655422,-1.648326,-2.085939,-1.539612,...,-1.443666,2.201572,-0.476503,-0.113778,-0.118523,-0.031869,-0.006247,-0.00255,-0.005981,-1.311051
3,100006,0,0.720391,-0.717914,-0.664531,-0.653139,-0.175535,-0.635708,0.378604,-0.540733,...,-0.553674,0.479660,-0.476503,-0.113778,8.437188,-0.031869,-0.006247,-0.00255,-0.005981,-0.015781
4,100007,0,-1.388135,-0.717914,-0.664531,-0.653139,-0.394442,0.042354,-0.196589,0.214928,...,0.526336,-0.481236,1.595372,-0.113778,-0.118523,-0.031869,-0.006247,-0.00255,-0.005981,1.433840


In [12]:
# indexを設定
#df_train_origin = df_train_origin.set_index('SK_ID_CURR')
#df_test_origin = df_test_origin.set_index('SK_ID_CURR')
#df_train_origin.head()

,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,-1.388135,-0.717914,-0.664531,-0.653139,0.647368,-0.284044,0.028202,-0.315489,0.046984,...,1.625994,-1.653296,-0.476503,-0.113778,-0.118523,-0.031869,-0.006247,-0.00255,-0.005981,-0.118159
100003,0,0.720391,-0.717914,1.504820,-0.653139,1.212877,1.495411,0.744282,1.406475,-1.730410,...,0.360092,0.055439,-0.476503,-0.113778,-0.118523,-0.031869,-0.006247,-0.00255,-0.005981,0.298617
100004,0,-1.388135,1.392925,-0.664531,-0.653139,-1.655422,-1.648326,-2.085939,-1.539612,-0.863345,...,-1.443666,2.201572,-0.476503,-0.113778,-0.118523,-0.031869,-0.006247,-0.00255,-0.005981,-1.311051
100006,0,0.720391,-0.717914,-0.664531,-0.653139,-0.175535,-0.635708,0.378604,-0.540733,-1.112140,...,-0.553674,0.479660,-0.476503,-0.113778,8.437188,-0.031869,-0.006247,-0.00255,-0.005981,-0.015781
100007,0,-1.388135,-0.717914,-0.664531,-0.653139,-0.394442,0.042354,-0.196589,0.214928,0.799861,...,0.526336,-0.481236,1.595372,-0.113778,-0.118523,-0.031869,-0.006247,-0.00255,-0.005981,1.433840


In [13]:
## 標準化
#for column in encoding_columns:
#    num_cols = []
#    num_cols.append(column)
#    result = modules.process_standardization(df_train_origin, df_test_origin, num_cols)
#        
#df_train_origin.head()

,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,-1.388135,-0.717914,-0.664531,-0.653139,0.647368,-0.284044,0.028202,-0.315489,0.046984,...,1.625994,-1.653296,-0.476503,-0.113778,-0.118523,-0.031869,-0.006247,-0.00255,-0.005981,-0.118159
100003,0,0.720391,-0.717914,1.504820,-0.653139,1.212877,1.495411,0.744282,1.406475,-1.730410,...,0.360092,0.055439,-0.476503,-0.113778,-0.118523,-0.031869,-0.006247,-0.00255,-0.005981,0.298617
100004,0,-1.388135,1.392925,-0.664531,-0.653139,-1.655422,-1.648326,-2.085939,-1.539612,-0.863345,...,-1.443666,2.201572,-0.476503,-0.113778,-0.118523,-0.031869,-0.006247,-0.00255,-0.005981,-1.311051
100006,0,0.720391,-0.717914,-0.664531,-0.653139,-0.175535,-0.635708,0.378604,-0.540733,-1.112140,...,-0.553674,0.479660,-0.476503,-0.113778,8.437188,-0.031869,-0.006247,-0.00255,-0.005981,-0.015781
100007,0,-1.388135,-0.717914,-0.664531,-0.653139,-0.394442,0.042354,-0.196589,0.214928,0.799861,...,0.526336,-0.481236,1.595372,-0.113778,-0.118523,-0.031869,-0.006247,-0.00255,-0.005981,1.433840


In [14]:
### indexを削除
#df_train_origin=df_train_origin.reset_index()
#df_test_origin=df_test_origin.reset_index()
#df_train_origin.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
0,100002,1,-1.388135,-0.717914,-0.664531,-0.653139,0.647368,-0.284044,0.028202,-0.315489,...,1.625994,-1.653296,-0.476503,-0.113778,-0.118523,-0.031869,-0.006247,-0.00255,-0.005981,-0.118159
1,100003,0,0.720391,-0.717914,1.504820,-0.653139,1.212877,1.495411,0.744282,1.406475,...,0.360092,0.055439,-0.476503,-0.113778,-0.118523,-0.031869,-0.006247,-0.00255,-0.005981,0.298617
2,100004,0,-1.388135,1.392925,-0.664531,-0.653139,-1.655422,-1.648326,-2.085939,-1.539612,...,-1.443666,2.201572,-0.476503,-0.113778,-0.118523,-0.031869,-0.006247,-0.00255,-0.005981,-1.311051
3,100006,0,0.720391,-0.717914,-0.664531,-0.653139,-0.175535,-0.635708,0.378604,-0.540733,...,-0.553674,0.479660,-0.476503,-0.113778,8.437188,-0.031869,-0.006247,-0.00255,-0.005981,-0.015781
4,100007,0,-1.388135,-0.717914,-0.664531,-0.653139,-0.394442,0.042354,-0.196589,0.214928,...,0.526336,-0.481236,1.595372,-0.113778,-0.118523,-0.031869,-0.006247,-0.00255,-0.005981,1.433840


In [16]:
df_train_origin = baseline.reduce_mem_usage(df_train_origin)
df_test_origin = baseline.reduce_mem_usage(df_test_origin)
df_train_origin.head()

Memory usage of dataframe is 7965.09 MB
Memory usage of optimization is 4175076975.00 MB
Decreased by -52417118.6%
Memory usage of dataframe is 1262.19 MB
Memory usage of optimization is 661748672.00 MB
Decreased by -52428705.1%


,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
0,100002,1,-1.387695,-0.717773,-0.664551,-0.65332,0.647461,-0.283936,0.028198,-0.315430,...,1.625977,-1.653320,-0.476562,-0.11377,-0.11853,-0.03186,-0.006248,-0.00255,-0.005981,-0.118164
1,100003,0,0.720215,-0.717773,1.504883,-0.65332,1.212891,1.495117,0.744141,1.406250,...,0.360107,0.055450,-0.476562,-0.11377,-0.11853,-0.03186,-0.006248,-0.00255,-0.005981,0.298584
2,100004,0,-1.387695,1.392578,-0.664551,-0.65332,-1.655273,-1.648438,-2.085938,-1.540039,...,-1.443359,2.201172,-0.476562,-0.11377,-0.11853,-0.03186,-0.006248,-0.00255,-0.005981,-1.311523
3,100006,0,0.720215,-0.717773,-0.664551,-0.65332,-0.175537,-0.635742,0.378662,-0.540527,...,-0.553711,0.479736,-0.476562,-0.11377,8.43750,-0.03186,-0.006248,-0.00255,-0.005981,-0.015778
4,100007,0,-1.387695,-0.717773,-0.664551,-0.65332,-0.394531,0.042358,-0.196533,0.214966,...,0.526367,-0.481201,1.595703,-0.11377,-0.11853,-0.03186,-0.006248,-0.00255,-0.005981,1.433594


In [17]:
df_train = df_train_origin.copy()
df_test = df_test_origin.copy()
df_train.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
0,100002,1,-1.387695,-0.717773,-0.664551,-0.65332,0.647461,-0.283936,0.028198,-0.315430,...,1.625977,-1.653320,-0.476562,-0.11377,-0.11853,-0.03186,-0.006248,-0.00255,-0.005981,-0.118164
1,100003,0,0.720215,-0.717773,1.504883,-0.65332,1.212891,1.495117,0.744141,1.406250,...,0.360107,0.055450,-0.476562,-0.11377,-0.11853,-0.03186,-0.006248,-0.00255,-0.005981,0.298584
2,100004,0,-1.387695,1.392578,-0.664551,-0.65332,-1.655273,-1.648438,-2.085938,-1.540039,...,-1.443359,2.201172,-0.476562,-0.11377,-0.11853,-0.03186,-0.006248,-0.00255,-0.005981,-1.311523
3,100006,0,0.720215,-0.717773,-0.664551,-0.65332,-0.175537,-0.635742,0.378662,-0.540527,...,-0.553711,0.479736,-0.476562,-0.11377,8.43750,-0.03186,-0.006248,-0.00255,-0.005981,-0.015778
4,100007,0,-1.387695,-0.717773,-0.664551,-0.65332,-0.394531,0.042358,-0.196533,0.214966,...,0.526367,-0.481201,1.595703,-0.11377,-0.11853,-0.03186,-0.006248,-0.00255,-0.005981,1.433594


## 学習(#001)

In [18]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [19]:
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 3393) (61503, 3393)
[100]	training's auc: 0.811063	valid_1's auc: 0.778075
[200]	training's auc: 0.845576	valid_1's auc: 0.785386
[300]	training's auc: 0.870394	valid_1's auc: 0.787129
[400]	training's auc: 0.890369	valid_1's auc: 0.787755
[500]	training's auc: 0.906558	valid_1's auc: 0.787983
[600]	training's auc: 0.920601	valid_1's auc: 0.787747
[auc] tr:0.906874, va0.7881
-------------------- 1 --------------------
(246009, 3393) (61502, 3393)
[100]	training's auc: 0.81071	valid_1's auc: 0.781851
[200]	training's auc: 0.845063	valid_1's auc: 0.790146
[300]	training's auc: 0.87055	valid_1's auc: 0.79169
[400]	training's auc: 0.891048	valid_1's auc: 0.791895
[500]	training's auc: 0.907988	valid_1's auc: 0.792004
[auc] tr:0.904772, va0.7921
-------------------- 2 --------------------
(246009, 3393) (61502, 3393)
[100]	training's auc: 0.810418	valid_1's auc: 0.781908
[200]	training's auc: 0.844633	valid_1's auc: 0.789486
[300]	trainin

In [20]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
3385,app_NEW_EXT_SOURCES_MEAN,112749.838874,1036.281037
3360,app_AMT_CREDIT_PER_AMT_ANNUITY,7489.275394,703.606336
3370,app_DAYS_EMPLOYED_div_DAYS_BIRTH,4918.332694,549.084802
2404,DAYS_BIRTH,4806.900466,383.957265
3361,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,4695.325947,326.364922
3373,app_EXT_SOURCE_1_PER_DAYS_BIRTH,4428.443591,289.186320
3379,app_EXT_SOURCE_2_times_EXT_SOURCE_3,4125.406948,342.325878
571,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_D...,4038.673796,847.418263
2505,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,3874.134936,252.294262
3359,app_AMT_CREDIT_AMT_GOODS_PRICE,3817.883008,240.339228


### 変数選択001

In [21]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 826147475224061181
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9374597120
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11714727779472560353
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:22:00.0, compute capability: 7.5"
 xla_global_id: 416903419,
 name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 1720110286
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 2309915562146741646
 physical_device_desc: "device: 1, name: NVIDIA RTX A2000 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
 xla_global_id: 2144165316]

In [22]:
import torch
# GPUの名称、CUDA Compute Capabilityを取得
print(torch.cuda.get_device_name())

NVIDIA RTX A2000 Laptop GPU


In [23]:
len(df_train.columns)

3395

In [24]:
length = int(len(df_train.columns) * 0.95)
length

3225

In [25]:
cols = list(imp.sort_values("imp", ascending=False)['col'][:length])
if 'TARGET' not in cols:
    print('aa')
    
if 'SK_ID_CURR' not in cols:
    print('bb')
    
train_columns = cols.copy()
train_columns.append('SK_ID_CURR')
train_columns.append('TARGET')

test_columns = cols.copy()
test_columns.append('SK_ID_CURR')

aa
bb


In [26]:
del df_train
del df_test
gc.collect()
df_train = df_train_origin[train_columns].copy()
df_test = df_test_origin[test_columns].copy()
df_train.head()

,app_NEW_EXT_SOURCES_MEAN,app_AMT_CREDIT_PER_AMT_ANNUITY,app_DAYS_EMPLOYED_div_DAYS_BIRTH,DAYS_BIRTH,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,app_EXT_SOURCE_1_PER_DAYS_BIRTH,app_EXT_SOURCE_2_times_EXT_SOURCE_3,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_DEBT_MIN,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,app_AMT_CREDIT_AMT_GOODS_PRICE,...,CLOSED_First_status_2_MEAN_SUM,CLOSED_First_status_2_MEAN_SIZE,CLOSED_First_status_2_MEAN_MIN,CLOSED_First_status_2_MEAN_MEAN,CLOSED_First_status_2_MEAN_MAX,CLOSED_First_status_1_MEAN_VAR,CLOSED_First_status_1_MEAN_SUM,CLOSED_DPD_5_Month_SIZE_MIN,SK_ID_CURR,TARGET
0,-2.046875,-0.596191,-0.849609,1.597656,0.388672,0.141479,-1.687500,-0.399170,-1.705078,-0.032013,...,-0.348633,0.537109,0.0,-1.209961,-1.209961,-1.374023,-0.349121,-1.388672,100002,1
1,-0.362793,1.686523,-0.817871,-0.222778,0.290283,-0.532715,1.546875,-0.328125,-0.032410,1.289062,...,-0.348633,0.062012,0.0,0.826172,0.826172,0.727539,-0.349121,0.787598,100003,0
2,0.889648,-0.109741,-1.407227,-0.716797,-1.044922,0.843262,0.272705,1.808594,-0.240479,-0.790039,...,-0.348633,-0.225464,0.0,0.826172,0.826172,0.727539,-0.349121,0.787598,100004,0
3,0.950195,-1.558594,-0.159912,-0.708008,-0.514648,0.843262,1.546875,-0.403564,-0.032410,-0.557129,...,-0.348633,-1.672852,0.0,-1.209961,-1.209961,-1.374023,-0.349121,-1.636719,100006,0
4,-1.232422,0.326172,-0.206299,-0.901367,-1.044922,0.843262,1.546875,1.808594,0.416260,-0.790039,...,-0.348633,-0.689453,0.0,0.826172,0.826172,0.727539,-0.349121,0.787598,100007,0


In [27]:
len(df_train.columns)

3227

In [28]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [29]:
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 3225) (61503, 3225)
[100]	training's auc: 0.8106	valid_1's auc: 0.77906
[200]	training's auc: 0.845054	valid_1's auc: 0.786338
[300]	training's auc: 0.870691	valid_1's auc: 0.787157
[400]	training's auc: 0.890334	valid_1's auc: 0.787371
[500]	training's auc: 0.907336	valid_1's auc: 0.787757
[auc] tr:0.905211, va0.7878
-------------------- 1 --------------------
(246009, 3225) (61502, 3225)
[100]	training's auc: 0.810094	valid_1's auc: 0.781603
[200]	training's auc: 0.844448	valid_1's auc: 0.789501
[300]	training's auc: 0.869323	valid_1's auc: 0.791097
[400]	training's auc: 0.889406	valid_1's auc: 0.791552
[500]	training's auc: 0.906406	valid_1's auc: 0.792093
[600]	training's auc: 0.920161	valid_1's auc: 0.791782
[auc] tr:0.904732, va0.7922
-------------------- 2 --------------------
(246009, 3225) (61502, 3225)
[100]	training's auc: 0.810614	valid_1's auc: 0.782918
[200]	training's auc: 0.844827	valid_1's auc: 0.78936
[300]	training

In [30]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
3217,app_NEW_EXT_SOURCES_MEAN,112756.944371,1052.305870
3192,app_AMT_CREDIT_PER_AMT_ANNUITY,7487.884074,737.791562
3202,app_DAYS_EMPLOYED_div_DAYS_BIRTH,4852.509812,367.476512
2236,DAYS_BIRTH,4679.423087,288.932013
3193,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,4676.166401,328.163866
3205,app_EXT_SOURCE_1_PER_DAYS_BIRTH,4354.403611,206.860218
3211,app_EXT_SOURCE_2_times_EXT_SOURCE_3,4180.735569,309.404271
571,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_D...,4042.792746,1005.117025
3191,app_AMT_CREDIT_AMT_GOODS_PRICE,3702.019805,355.733876
2337,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,3627.968131,348.630173


In [31]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [32]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [33]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_LgbFeatureSelection1.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.065215
1,100005,0.114357
2,100013,0.043379
3,100028,0.036281
4,100038,0.164656


### 変数選択002

In [35]:
len(df_train.columns)

3227

In [36]:
length = int(len(df_train.columns) * 0.95)
length

3065

In [37]:
cols = list(imp.sort_values("imp", ascending=False)['col'][:length])
if 'TARGET' not in cols:
    print('aa')
    
if 'SK_ID_CURR' not in cols:
    print('bb')
    
train_columns = cols.copy()
train_columns.append('SK_ID_CURR')
train_columns.append('TARGET')

test_columns = cols.copy()
test_columns.append('SK_ID_CURR')

aa
bb


In [38]:
del df_train
del df_test
gc.collect()
df_train = df_train_origin[train_columns].copy()
df_test = df_test_origin[test_columns].copy()
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 3596.90 MB
Memory usage of optimization is 3771622543.00 MB
Decreased by -104857500.0%
Memory usage of dataframe is 570.10 MB
Memory usage of optimization is 597796544.00 MB
Decreased by -104857500.0%


,app_NEW_EXT_SOURCES_MEAN,app_AMT_CREDIT_PER_AMT_ANNUITY,app_DAYS_EMPLOYED_div_DAYS_BIRTH,DAYS_BIRTH,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,app_EXT_SOURCE_1_PER_DAYS_BIRTH,app_EXT_SOURCE_2_times_EXT_SOURCE_3,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_DEBT_MIN,app_AMT_CREDIT_AMT_GOODS_PRICE,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,...,APPROVED_NAME_GOODS_CATEGORY_House_Construction_MEAN,APPROVED_NAME_GOODS_CATEGORY_Insurance_MEAN,CARD_card_missing_SIZE,CARD_card_missing_MIN,APPROVED_NAME_GOODS_CATEGORY_Medical_Supplies_MEAN,CARD_card_SK_DPD__SK_DPD_DEF_SUM,CARD_card_SK_DPD__SK_DPD_DEF_SIZE,CARD_card_SK_DPD__SK_DPD_DEF_MIN,SK_ID_CURR,TARGET
0,-2.046875,-0.596191,-0.849609,1.597656,0.388672,0.141479,-1.687500,-0.399170,-0.032013,-1.705078,...,0.0,-0.058105,-0.624512,-0.304688,-0.111511,-0.14624,-0.624512,0.0,100002,1
1,-0.362793,1.686523,-0.817871,-0.222778,0.290283,-0.532715,1.546875,-0.328125,1.289062,-0.032410,...,0.0,-0.058105,-0.624512,-0.304688,-0.111511,-0.14624,-0.624512,0.0,100003,0
2,0.889648,-0.109741,-1.407227,-0.716797,-1.044922,0.843262,0.272705,1.808594,-0.790039,-0.240479,...,0.0,-0.058105,-0.624512,-0.304688,-0.111511,-0.14624,-0.624512,0.0,100004,0
3,0.950195,-1.558594,-0.159912,-0.708008,-0.514648,0.843262,1.546875,-0.403564,-0.557129,-0.032410,...,0.0,-0.058105,1.372070,3.283203,-0.111511,-0.14624,1.372070,0.0,100006,0
4,-1.232422,0.326172,-0.206299,-0.901367,-1.044922,0.843262,1.546875,1.808594,-0.790039,0.416260,...,0.0,-0.058105,-0.624512,-0.304688,-0.111511,-0.14624,-0.624512,0.0,100007,0


In [39]:
len(df_train.columns)

3067

In [40]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [41]:
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 3065) (61503, 3065)
[100]	training's auc: 0.811004	valid_1's auc: 0.779219
[200]	training's auc: 0.845838	valid_1's auc: 0.786153
[300]	training's auc: 0.87024	valid_1's auc: 0.786773
[400]	training's auc: 0.890752	valid_1's auc: 0.787098
[500]	training's auc: 0.907669	valid_1's auc: 0.78682
[auc] tr:0.891556, va0.7871
-------------------- 1 --------------------
(246009, 3065) (61502, 3065)
[100]	training's auc: 0.810581	valid_1's auc: 0.780285
[200]	training's auc: 0.845271	valid_1's auc: 0.789007
[300]	training's auc: 0.86943	valid_1's auc: 0.790632
[400]	training's auc: 0.889746	valid_1's auc: 0.790404
[auc] tr:0.877068, va0.7910
-------------------- 2 --------------------
(246009, 3065) (61502, 3065)
[100]	training's auc: 0.810822	valid_1's auc: 0.782273
[200]	training's auc: 0.844258	valid_1's auc: 0.787846
[300]	training's auc: 0.869228	valid_1's auc: 0.788621
[auc] tr:0.861413, va0.7894
-------------------- 3 -----------------

In [42]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
3057,app_NEW_EXT_SOURCES_MEAN,112292.758584,1365.993599
3032,app_AMT_CREDIT_PER_AMT_ANNUITY,7209.941145,621.676627
3042,app_DAYS_EMPLOYED_div_DAYS_BIRTH,4691.080790,339.963041
3033,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,4643.394622,190.181651
2076,DAYS_BIRTH,4458.767223,343.809312
3045,app_EXT_SOURCE_1_PER_DAYS_BIRTH,4280.418418,135.668462
3051,app_EXT_SOURCE_2_times_EXT_SOURCE_3,4211.536550,325.136599
571,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_D...,3892.758780,952.717332
2177,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,3591.023136,142.727151
3031,app_AMT_CREDIT_AMT_GOODS_PRICE,3520.932335,425.428712


In [43]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [44]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [45]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_LgbFeatureSelection2.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.049913
1,100005,0.113793
2,100013,0.033444
3,100028,0.033927
4,100038,0.147003


### 変数選択003

In [46]:
len(df_train.columns)

3067

In [47]:
length = int(len(df_train.columns) * 0.95)
length

2913

In [48]:
cols = list(imp.sort_values("imp", ascending=False)['col'][:length])
if 'TARGET' not in cols:
    print('aa')
    
if 'SK_ID_CURR' not in cols:
    print('bb')
    
train_columns = cols.copy()
train_columns.append('SK_ID_CURR')
train_columns.append('TARGET')

test_columns = cols.copy()
test_columns.append('SK_ID_CURR')

aa
bb


In [49]:
del df_train
del df_test
gc.collect()
df_train = df_train_origin[train_columns].copy()
df_test = df_test_origin[test_columns].copy()
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 3418.59 MB
Memory usage of optimization is 3584655855.00 MB
Decreased by -104857500.0%
Memory usage of dataframe is 541.84 MB
Memory usage of optimization is 568160192.00 MB
Decreased by -104857500.0%


,app_NEW_EXT_SOURCES_MEAN,app_AMT_CREDIT_PER_AMT_ANNUITY,app_DAYS_EMPLOYED_div_DAYS_BIRTH,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,DAYS_BIRTH,app_EXT_SOURCE_1_PER_DAYS_BIRTH,app_EXT_SOURCE_2_times_EXT_SOURCE_3,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_DEBT_MIN,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,app_AMT_CREDIT_AMT_GOODS_PRICE,...,APPROVED_NAME_CLIENT_TYPE_XNA_MEAN,APPROVED_NAME_CONTRACT_STATUS_Approved_MEAN,CARD_card_AMT_BALANCE__AMT_RECIVABLE_SIZE,APPROVED_NAME_CONTRACT_STATUS_Canceled_MEAN,APPROVED_NAME_PAYMENT_TYPE_Non_cash_from_your_account_MEAN,CARD_CNT_DRAWINGS_POS_CURRENT_SIZE,APPROVED_NAME_PORTFOLIO_Cars_MEAN,APPROVED_NAME_PORTFOLIO_Cash_MEAN,SK_ID_CURR,TARGET
0,-2.046875,-0.596191,-0.849609,0.388672,1.597656,0.141479,-1.687500,-0.399170,-1.705078,-0.032013,...,-0.067993,0.151855,-0.624512,0.0,-0.143311,-0.624512,-0.062805,-0.851562,100002,1
1,-0.362793,1.686523,-0.817871,0.290283,-0.222778,-0.532715,1.546875,-0.328125,-0.032410,1.289062,...,-0.067993,0.151855,-0.624512,0.0,-0.143311,-0.624512,-0.062805,0.847656,100003,0
2,0.889648,-0.109741,-1.407227,-1.044922,-0.716797,0.843262,0.272705,1.808594,-0.240479,-0.790039,...,-0.067993,0.151855,-0.624512,0.0,-0.143311,-0.624512,-0.062805,-0.851562,100004,0
3,0.950195,-1.558594,-0.159912,-0.514648,-0.708008,0.843262,1.546875,-0.403564,-0.032410,-0.557129,...,-0.067993,0.151855,1.372070,0.0,-0.143311,1.372070,-0.062805,1.009766,100006,0
4,-1.232422,0.326172,-0.206299,-1.044922,-0.901367,0.843262,1.546875,1.808594,0.416260,-0.790039,...,-0.067993,0.151855,-0.624512,0.0,-0.143311,-0.624512,-0.062805,1.417969,100007,0


In [50]:
len(df_train.columns)

2915

In [51]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [52]:
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 2913) (61503, 2913)
[100]	training's auc: 0.810369	valid_1's auc: 0.778087
[200]	training's auc: 0.846031	valid_1's auc: 0.785216
[300]	training's auc: 0.870846	valid_1's auc: 0.786499
[400]	training's auc: 0.891174	valid_1's auc: 0.786649
[500]	training's auc: 0.907309	valid_1's auc: 0.786849
[600]	training's auc: 0.921029	valid_1's auc: 0.786877
[auc] tr:0.912433, va0.7871
-------------------- 1 --------------------
(246009, 2913) (61502, 2913)
[100]	training's auc: 0.810312	valid_1's auc: 0.780966
[200]	training's auc: 0.845586	valid_1's auc: 0.78946
[300]	training's auc: 0.870621	valid_1's auc: 0.790411
[400]	training's auc: 0.891072	valid_1's auc: 0.790794
[500]	training's auc: 0.908273	valid_1's auc: 0.79123
[600]	training's auc: 0.921892	valid_1's auc: 0.79141
[auc] tr:0.916070, va0.7916
-------------------- 2 --------------------
(246009, 2913) (61502, 2913)
[100]	training's auc: 0.810601	valid_1's auc: 0.782179
[200]	trainin

In [53]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
2905,app_NEW_EXT_SOURCES_MEAN,112691.181889,1614.131788
2880,app_AMT_CREDIT_PER_AMT_ANNUITY,7707.299501,876.897575
2890,app_DAYS_EMPLOYED_div_DAYS_BIRTH,5052.987576,370.887000
2881,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,4786.255717,201.930224
1924,DAYS_BIRTH,4695.102295,530.659042
2893,app_EXT_SOURCE_1_PER_DAYS_BIRTH,4386.581643,135.341260
2899,app_EXT_SOURCE_2_times_EXT_SOURCE_3,4186.838401,428.888168
570,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_D...,3920.993668,985.888260
2025,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,3645.489972,352.458365
2879,app_AMT_CREDIT_AMT_GOODS_PRICE,3642.181315,595.719345


In [54]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [55]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [56]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_LgbFeatureSelection3.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.058165
1,100005,0.107014
2,100013,0.025455
3,100028,0.038042
4,100038,0.162251


### 変数選択004

In [57]:
len(df_train.columns)

2915

In [58]:
length = int(len(df_train.columns) * 0.95)
length

2769

In [59]:
cols = list(imp.sort_values("imp", ascending=False)['col'][:length])
if 'TARGET' not in cols:
    print('aa')
    
if 'SK_ID_CURR' not in cols:
    print('bb')
    
train_columns = cols.copy()
train_columns.append('SK_ID_CURR')
train_columns.append('TARGET')

test_columns = cols.copy()
test_columns.append('SK_ID_CURR')

aa
bb


In [60]:
del df_train
del df_test
gc.collect()
df_train = df_train_origin[train_columns].copy()
df_test = df_test_origin[test_columns].copy()
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 3249.67 MB
Memory usage of optimization is 3407529519.00 MB
Decreased by -104857500.0%
Memory usage of dataframe is 515.06 MB
Memory usage of optimization is 540083648.00 MB
Decreased by -104857500.0%


,app_NEW_EXT_SOURCES_MEAN,app_AMT_CREDIT_PER_AMT_ANNUITY,app_DAYS_EMPLOYED_div_DAYS_BIRTH,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,DAYS_BIRTH,app_EXT_SOURCE_1_PER_DAYS_BIRTH,app_EXT_SOURCE_2_times_EXT_SOURCE_3,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_DEBT_MIN,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,app_AMT_CREDIT_AMT_GOODS_PRICE,...,CARD_card_missing_SIZE,CARD_card_missing_MIN,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_MEAN,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_MEAN,CARD_card_SK_DPD__SK_DPD_DEF_SUM,CARD_card_SK_DPD__SK_DPD_DEF_SIZE,CARD_card_SK_DPD__SK_DPD_DEF_MIN,CARD_card_SK_DPD__MONTHS_BALANCE_SIZE,SK_ID_CURR,TARGET
0,-2.046875,-0.596191,-0.849609,0.388672,1.597656,0.141479,-1.687500,-0.399170,-1.705078,-0.032013,...,-0.624512,-0.304688,-0.062103,-0.074951,-0.14624,-0.624512,0.0,-0.624512,100002,1
1,-0.362793,1.686523,-0.817871,0.290283,-0.222778,-0.532715,1.546875,-0.328125,-0.032410,1.289062,...,-0.624512,-0.304688,-0.062103,-0.074951,-0.14624,-0.624512,0.0,-0.624512,100003,0
2,0.889648,-0.109741,-1.407227,-1.044922,-0.716797,0.843262,0.272705,1.808594,-0.240479,-0.790039,...,-0.624512,-0.304688,-0.062103,-0.074951,-0.14624,-0.624512,0.0,-0.624512,100004,0
3,0.950195,-1.558594,-0.159912,-0.514648,-0.708008,0.843262,1.546875,-0.403564,-0.032410,-0.557129,...,1.372070,3.283203,-0.062103,-0.074951,-0.14624,1.372070,0.0,1.372070,100006,0
4,-1.232422,0.326172,-0.206299,-1.044922,-0.901367,0.843262,1.546875,1.808594,0.416260,-0.790039,...,-0.624512,-0.304688,-0.062103,-0.074951,-0.14624,-0.624512,0.0,-0.624512,100007,0


In [61]:
len(df_train.columns)

2771

In [62]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [63]:
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 2769) (61503, 2769)
[100]	training's auc: 0.810761	valid_1's auc: 0.779093
[200]	training's auc: 0.845853	valid_1's auc: 0.786037
[300]	training's auc: 0.871257	valid_1's auc: 0.78713
[400]	training's auc: 0.892257	valid_1's auc: 0.787479
[500]	training's auc: 0.90831	valid_1's auc: 0.787349
[auc] tr:0.898554, va0.7877
-------------------- 1 --------------------
(246009, 2769) (61502, 2769)
[100]	training's auc: 0.81032	valid_1's auc: 0.781007
[200]	training's auc: 0.845009	valid_1's auc: 0.788615
[300]	training's auc: 0.871395	valid_1's auc: 0.790801
[400]	training's auc: 0.891877	valid_1's auc: 0.791014
[500]	training's auc: 0.908037	valid_1's auc: 0.791273
[auc] tr:0.898470, va0.7915
-------------------- 2 --------------------
(246009, 2769) (61502, 2769)
[100]	training's auc: 0.810648	valid_1's auc: 0.781912
[200]	training's auc: 0.843786	valid_1's auc: 0.788499
[300]	training's auc: 0.869493	valid_1's auc: 0.789955
[400]	trainin

In [64]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
2761,app_NEW_EXT_SOURCES_MEAN,112530.356757,1273.530731
2736,app_AMT_CREDIT_PER_AMT_ANNUITY,7268.677735,614.837401
2737,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,4843.080393,213.003679
2746,app_DAYS_EMPLOYED_div_DAYS_BIRTH,4793.505998,443.883014
1780,DAYS_BIRTH,4729.947327,412.400675
2749,app_EXT_SOURCE_1_PER_DAYS_BIRTH,4383.117934,225.318900
2755,app_EXT_SOURCE_2_times_EXT_SOURCE_3,4332.918330,333.674944
557,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_D...,3962.051017,1064.892221
1881,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,3655.395834,218.406821
1776,CODE_GENDER,3544.642767,256.030932


In [65]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [66]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [67]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_LgbFeatureSelection4.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.050067
1,100005,0.111334
2,100013,0.035095
3,100028,0.039011
4,100038,0.160877


### 変数選択005

In [68]:
len(df_train.columns)

2771

In [69]:
length = int(len(df_train.columns) * 0.95)
length

2632

In [70]:
cols = list(imp.sort_values("imp", ascending=False)['col'][:length])
if 'TARGET' not in cols:
    print('aa')
    
if 'SK_ID_CURR' not in cols:
    print('bb')
    
train_columns = cols.copy()
train_columns.append('SK_ID_CURR')
train_columns.append('TARGET')

test_columns = cols.copy()
test_columns.append('SK_ID_CURR')

aa
bb


In [71]:
del df_train
del df_test
gc.collect()
df_train = df_train_origin[train_columns].copy()
df_test = df_test_origin[test_columns].copy()
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 3088.96 MB
Memory usage of optimization is 3239013491.00 MB
Decreased by -104857500.0%
Memory usage of dataframe is 489.59 MB
Memory usage of optimization is 513371936.00 MB
Decreased by -104857500.0%


,app_NEW_EXT_SOURCES_MEAN,app_AMT_CREDIT_PER_AMT_ANNUITY,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,app_DAYS_EMPLOYED_div_DAYS_BIRTH,DAYS_BIRTH,app_EXT_SOURCE_1_PER_DAYS_BIRTH,app_EXT_SOURCE_2_times_EXT_SOURCE_3,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_DEBT_MIN,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,CODE_GENDER,...,BURO_DPD_0_cnt_MIN_SIZE,BURO_DPD_0_cnt_SIZE_MAX,INS_NUM_INSTALMENT_NUMBER_SIZE,BURO_DPD_0_cnt_SIZE_MEAN,BURO_DPD_0_cnt_SIZE_MIN,BURO_DPD_0_cnt_SIZE_SIZE,BURO_DPD_0_cnt_SIZE_VAR,BURO_DPD_1_Month_MAX_MAX,SK_ID_CURR,TARGET
0,-2.046875,-0.596191,0.388672,-0.849609,1.597656,0.141479,-1.687500,-0.399170,-1.705078,-1.387695,...,0.910156,-1.052734,-0.216431,-1.050781,-1.177734,0.910156,-1.208008,-1.753906,100002,1
1,-0.362793,1.686523,0.290283,-0.817871,-0.222778,-0.532715,1.546875,-0.328125,-0.032410,0.720215,...,0.139893,0.798340,0.019257,0.836426,0.852539,0.139893,0.784668,0.563477,100003,0
2,0.889648,-0.109741,-1.044922,-1.407227,-0.716797,0.843262,0.272705,1.808594,-0.240479,-1.387695,...,-0.474365,0.798340,-1.433594,0.836426,0.852539,-0.474365,0.784668,0.563477,100004,0
3,0.950195,-1.558594,-0.514648,-0.159912,-0.708008,0.843262,1.546875,-0.403564,-0.032410,0.720215,...,-1.640625,-1.750000,-0.356689,-1.669922,-1.535156,-1.640625,-1.599609,-1.784180,100006,0
4,-1.232422,0.326172,-1.044922,-0.206299,-0.901367,0.843262,1.546875,1.808594,0.416260,-1.387695,...,-0.928223,0.798340,0.973145,0.836426,0.852539,-0.928223,0.784668,0.563477,100007,0


In [72]:
len(df_train.columns)

2634

In [73]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [74]:
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 2632) (61503, 2632)
[100]	training's auc: 0.810961	valid_1's auc: 0.778297
[200]	training's auc: 0.844892	valid_1's auc: 0.785253
[300]	training's auc: 0.870685	valid_1's auc: 0.785673
[400]	training's auc: 0.89136	valid_1's auc: 0.785699
[500]	training's auc: 0.907742	valid_1's auc: 0.785674
[auc] tr:0.898610, va0.7860
-------------------- 1 --------------------
(246009, 2632) (61502, 2632)
[100]	training's auc: 0.810714	valid_1's auc: 0.781454
[200]	training's auc: 0.84498	valid_1's auc: 0.789347
[300]	training's auc: 0.870158	valid_1's auc: 0.790885
[auc] tr:0.867537, va0.7911
-------------------- 2 --------------------
(246009, 2632) (61502, 2632)
[100]	training's auc: 0.810627	valid_1's auc: 0.782893
[200]	training's auc: 0.844845	valid_1's auc: 0.789343
[300]	training's auc: 0.870742	valid_1's auc: 0.788612
[auc] tr:0.850295, va0.7897
-------------------- 3 --------------------
(246009, 2632) (61502, 2632)
[100]	training's auc:

In [75]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
2624,app_NEW_EXT_SOURCES_MEAN,112318.420307,1467.089785
2599,app_AMT_CREDIT_PER_AMT_ANNUITY,7066.882400,858.815521
2600,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,4730.801845,171.508571
2609,app_DAYS_EMPLOYED_div_DAYS_BIRTH,4720.208463,355.570755
1672,DAYS_BIRTH,4302.413911,443.314688
2612,app_EXT_SOURCE_1_PER_DAYS_BIRTH,4256.108204,174.767684
2618,app_EXT_SOURCE_2_times_EXT_SOURCE_3,4233.982469,287.685777
557,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_D...,3892.784861,1043.195804
1758,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,3569.306157,201.840315
1668,CODE_GENDER,3486.647041,328.124916


In [76]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [77]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [78]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_LgbFeatureSelection5.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.053575
1,100005,0.109456
2,100013,0.028859
3,100028,0.037416
4,100038,0.165993


### 変数選択006

In [79]:
len(df_train.columns)

2634

In [80]:
length = int(len(df_train.columns) * 0.95)
length

2502

In [81]:
cols = list(imp.sort_values("imp", ascending=False)['col'][:length])
if 'TARGET' not in cols:
    print('aa')
    
if 'SK_ID_CURR' not in cols:
    print('bb')
    
train_columns = cols.copy()
train_columns.append('SK_ID_CURR')
train_columns.append('TARGET')

test_columns = cols.copy()
test_columns.append('SK_ID_CURR')

aa
bb


In [82]:
del df_train
del df_test
gc.collect()
df_train = df_train_origin[train_columns].copy()
df_test = df_test_origin[test_columns].copy()
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 2936.47 MB
Memory usage of optimization is 3079107771.00 MB
Decreased by -104857500.0%
Memory usage of dataframe is 465.42 MB
Memory usage of optimization is 488025056.00 MB
Decreased by -104857500.0%


,app_NEW_EXT_SOURCES_MEAN,app_AMT_CREDIT_PER_AMT_ANNUITY,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,app_DAYS_EMPLOYED_div_DAYS_BIRTH,DAYS_BIRTH,app_EXT_SOURCE_1_PER_DAYS_BIRTH,app_EXT_SOURCE_2_times_EXT_SOURCE_3,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_DEBT_MIN,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,CODE_GENDER,...,CLOSED_First_status_X_MEAN_SUM,CLOSED_STATUS_4_MEAN_MEAN,CLOSED_STATUS_5_MEAN_MAX,CLOSED_STATUS_X_MEAN_SIZE,CLOSED_When_closed_MAX_MAX,CLOSED_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_DEBT_SIZE,ACTIVE_SK_ID_CURR_MAX,ACTIVE_SK_ID_CURR_MEAN,SK_ID_CURR,TARGET
0,-2.046875,-0.596191,0.388672,-0.849609,1.597656,0.141479,-1.687500,-0.399170,-1.705078,-1.387695,...,1.660156,-1.210938,-1.213867,0.537109,-1.069336,0.537109,-0.575195,-0.575195,100002,1
1,-0.362793,1.686523,0.290283,-0.817871,-0.222778,-0.532715,1.546875,-0.328125,-0.032410,0.720215,...,-0.602051,0.826172,0.825684,0.062012,0.694336,0.062012,-0.575195,-0.575195,100003,0
2,0.889648,-0.109741,-1.044922,-1.407227,-0.716797,0.843262,0.272705,1.808594,-0.240479,-1.387695,...,-0.602051,0.826172,0.825684,-0.225464,0.694336,-0.225464,1.174805,1.174805,100004,0
3,0.950195,-1.558594,-0.514648,-0.159912,-0.708008,0.843262,1.546875,-0.403564,-0.032410,0.720215,...,-0.602051,-1.210938,-1.213867,-1.672852,-1.855469,-1.672852,-2.162109,-2.162109,100006,0
4,-1.232422,0.326172,-1.044922,-0.206299,-0.901367,0.843262,1.546875,1.808594,0.416260,-1.387695,...,-0.602051,0.826172,0.825684,-0.689453,0.694336,-0.689453,1.174805,1.174805,100007,0


In [83]:
len(df_train.columns)

2504

In [84]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [85]:
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 2502) (61503, 2502)
[100]	training's auc: 0.810827	valid_1's auc: 0.77864
[200]	training's auc: 0.844749	valid_1's auc: 0.785635
[300]	training's auc: 0.871199	valid_1's auc: 0.786904
[400]	training's auc: 0.891291	valid_1's auc: 0.787125
[500]	training's auc: 0.907699	valid_1's auc: 0.787081
[auc] tr:0.900190, va0.7873
-------------------- 1 --------------------
(246009, 2502) (61502, 2502)
[100]	training's auc: 0.810881	valid_1's auc: 0.781355
[200]	training's auc: 0.844697	valid_1's auc: 0.789632
[300]	training's auc: 0.869827	valid_1's auc: 0.791719
[400]	training's auc: 0.889844	valid_1's auc: 0.79134
[auc] tr:0.870096, va0.7918
-------------------- 2 --------------------
(246009, 2502) (61502, 2502)
[100]	training's auc: 0.810682	valid_1's auc: 0.781927
[200]	training's auc: 0.843721	valid_1's auc: 0.7884
[300]	training's auc: 0.870501	valid_1's auc: 0.789364
[400]	training's auc: 0.890723	valid_1's auc: 0.790102
[500]	training

In [86]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
2494,app_NEW_EXT_SOURCES_MEAN,112594.159842,1062.298246
2469,app_AMT_CREDIT_PER_AMT_ANNUITY,7070.700012,660.486632
2479,app_DAYS_EMPLOYED_div_DAYS_BIRTH,4713.262020,372.757929
2470,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,4700.442400,296.283715
1547,DAYS_BIRTH,4437.626697,190.493476
2482,app_EXT_SOURCE_1_PER_DAYS_BIRTH,4235.597968,140.174913
2488,app_EXT_SOURCE_2_times_EXT_SOURCE_3,4124.734429,407.711342
468,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_D...,3890.961292,972.274372
1628,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,3549.387680,121.428487
1543,CODE_GENDER,3470.553158,273.056304


In [87]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [88]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [89]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_LgbFeatureSelection6.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.062555
1,100005,0.108517
2,100013,0.028590
3,100028,0.035915
4,100038,0.172655


### 変数選択007

In [90]:
len(df_train.columns)

2504

In [91]:
length = int(len(df_train.columns) * 0.95)
length

2378

In [92]:
cols = list(imp.sort_values("imp", ascending=False)['col'][:length])
if 'TARGET' not in cols:
    print('aa')
    
if 'SK_ID_CURR' not in cols:
    print('bb')
    
train_columns = cols.copy()
train_columns.append('SK_ID_CURR')
train_columns.append('TARGET')

test_columns = cols.copy()
test_columns.append('SK_ID_CURR')

aa
bb


In [93]:
del df_train
del df_test
gc.collect()
df_train = df_train_origin[train_columns].copy()
df_test = df_test_origin[test_columns].copy()
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 2791.01 MB
Memory usage of optimization is 2926582315.00 MB
Decreased by -104857500.0%
Memory usage of dataframe is 442.36 MB
Memory usage of optimization is 463848032.00 MB
Decreased by -104857500.0%


,app_NEW_EXT_SOURCES_MEAN,app_AMT_CREDIT_PER_AMT_ANNUITY,app_DAYS_EMPLOYED_div_DAYS_BIRTH,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,DAYS_BIRTH,app_EXT_SOURCE_1_PER_DAYS_BIRTH,app_EXT_SOURCE_2_times_EXT_SOURCE_3,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_DEBT_MIN,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,CODE_GENDER,...,PREV_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_MEAN,ACTIVE_Last_status_2_MEAN_MEAN,POS_pos_CNT_INSTALMENT_more_CNT_INSTALMENT_FUTURE_SIZE,POS_pos_CNT_INSTALMENT_more_CNT_INSTALMENT_FUTURE_MIN,ACTIVE_Last_status_2_MEAN_MIN,BURO_DPD_0_cnt_MAX_VAR,BURO_DPD_0_cnt_MAX_SIZE,SK_ID_CURR,TARGET
0,-2.046875,-0.596191,-0.849609,0.388672,1.597656,0.141479,-1.687500,-0.399170,-1.705078,-1.387695,...,-0.039917,-0.04306,-1.246094,-0.118164,-0.098572,-1.246094,-1.052734,0.910156,100002,1
1,-0.362793,1.686523,-0.817871,0.290283,-0.222778,-0.532715,1.546875,-0.328125,-0.032410,0.720215,...,-0.039917,-0.04306,0.802734,0.298584,-0.098572,0.802734,0.784668,0.139893,100003,0
2,0.889648,-0.109741,-1.407227,-1.044922,-0.716797,0.843262,0.272705,1.808594,-0.240479,-1.387695,...,-0.039917,-0.04306,0.802734,-1.311523,-0.098572,0.802734,0.784668,-0.474365,100004,0
3,0.950195,-1.558594,-0.159912,-0.514648,-0.708008,0.843262,1.546875,-0.403564,-0.032410,0.720215,...,-0.039917,-0.04306,-1.246094,-0.015778,-0.098572,-1.246094,-1.459961,-1.640625,100006,0
4,-1.232422,0.326172,-0.206299,-1.044922,-0.901367,0.843262,1.546875,1.808594,0.416260,-1.387695,...,-0.039917,-0.04306,0.802734,1.433594,-0.098572,0.802734,0.784668,-0.928223,100007,0


In [94]:
len(df_train.columns)

2380

In [95]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [96]:
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 2378) (61503, 2378)
[100]	training's auc: 0.810551	valid_1's auc: 0.778121
[200]	training's auc: 0.845174	valid_1's auc: 0.785453
[300]	training's auc: 0.871612	valid_1's auc: 0.78606
[400]	training's auc: 0.891028	valid_1's auc: 0.786091
[auc] tr:0.870116, va0.7862
-------------------- 1 --------------------
(246009, 2378) (61502, 2378)
[100]	training's auc: 0.810518	valid_1's auc: 0.780544
[200]	training's auc: 0.844958	valid_1's auc: 0.788901
[300]	training's auc: 0.869878	valid_1's auc: 0.79001
[400]	training's auc: 0.89095	valid_1's auc: 0.790382
[auc] tr:0.887549, va0.7904
-------------------- 2 --------------------
(246009, 2378) (61502, 2378)
[100]	training's auc: 0.81044	valid_1's auc: 0.782284
[200]	training's auc: 0.844574	valid_1's auc: 0.789434
[300]	training's auc: 0.870371	valid_1's auc: 0.790855
[400]	training's auc: 0.890489	valid_1's auc: 0.791399
[auc] tr:0.878144, va0.7916
-------------------- 3 ------------------

In [97]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
2370,app_NEW_EXT_SOURCES_MEAN,112342.793222,1220.784440
2345,app_AMT_CREDIT_PER_AMT_ANNUITY,7490.641855,640.729723
2346,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,4865.910138,234.590595
2355,app_DAYS_EMPLOYED_div_DAYS_BIRTH,4815.924062,459.907400
1465,DAYS_BIRTH,4428.792413,364.491460
2358,app_EXT_SOURCE_1_PER_DAYS_BIRTH,4249.537229,189.112023
2364,app_EXT_SOURCE_2_times_EXT_SOURCE_3,4113.253618,247.044972
439,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_D...,3895.595921,994.100876
1546,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,3611.290333,287.951406
1461,CODE_GENDER,3542.792220,237.293004


In [98]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [99]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [100]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_LgbFeatureSelection7.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.057324
1,100005,0.128481
2,100013,0.032475
3,100028,0.034905
4,100038,0.177032


### 変数選択008

In [101]:
len(df_train.columns)

2380

In [102]:
length = int(len(df_train.columns) * 0.95)
length

2261

In [103]:
cols = list(imp.sort_values("imp", ascending=False)['col'][:length])
if 'TARGET' not in cols:
    print('aa')
    
if 'SK_ID_CURR' not in cols:
    print('bb')
    
train_columns = cols.copy()
train_columns.append('SK_ID_CURR')
train_columns.append('TARGET')

test_columns = cols.copy()
test_columns.append('SK_ID_CURR')

aa
bb


In [104]:
del df_train
del df_test
gc.collect()
df_train = df_train_origin[train_columns].copy()
df_test = df_test_origin[test_columns].copy()
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 2653.76 MB
Memory usage of optimization is 2782667167.00 MB
Decreased by -104857500.0%
Memory usage of dataframe is 420.60 MB
Memory usage of optimization is 441035840.00 MB
Decreased by -104857500.0%


,app_NEW_EXT_SOURCES_MEAN,app_AMT_CREDIT_PER_AMT_ANNUITY,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,app_DAYS_EMPLOYED_div_DAYS_BIRTH,DAYS_BIRTH,app_EXT_SOURCE_1_PER_DAYS_BIRTH,app_EXT_SOURCE_2_times_EXT_SOURCE_3,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_DEBT_MIN,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,CODE_GENDER,...,ACTIVE_DPD_4_Month_MIN_MIN,BURO_Month_closed_to_end_SIZE_VAR,ACTIVE_DPD_4_Month_MIN_MAX,ACTIVE_DPD_4_Month_MAX_VAR,ACTIVE_DPD_4_Month_MAX_SUM,ACTIVE_DPD_4_Month_MAX_SIZE,BURO_Non_zero_DPD_cnt_MAX_MAX,ACTIVE_DPD_4_Month_MAX_MIN,SK_ID_CURR,TARGET
0,-2.046875,-0.596191,0.388672,-0.849609,1.597656,0.141479,-1.687500,-0.399170,-1.705078,-1.387695,...,0.412109,-1.208008,0.412109,0.409424,-0.423340,-0.055267,-0.572266,0.412109,100002,1
1,-0.362793,1.686523,0.290283,-0.817871,-0.222778,-0.532715,1.546875,-0.328125,-0.032410,0.720215,...,0.412109,0.784668,0.412109,0.409424,-0.423340,-0.561523,0.883301,0.412109,100003,0
2,0.889648,-0.109741,-1.044922,-1.407227,-0.716797,0.843262,0.272705,1.808594,-0.240479,-1.387695,...,0.412109,0.784668,0.412109,0.409424,2.369141,1.738281,0.883301,0.412109,100004,0
3,0.950195,-1.558594,-0.514648,-0.159912,-0.708008,0.843262,1.546875,-0.403564,-0.032410,0.720215,...,-2.425781,-1.599609,-2.425781,-2.443359,-0.423340,-1.671875,-1.201172,-2.425781,100006,0
4,-1.232422,0.326172,-1.044922,-0.206299,-0.901367,0.843262,1.546875,1.808594,0.416260,-1.387695,...,0.412109,0.784668,0.412109,0.409424,2.369141,1.738281,0.883301,0.412109,100007,0


In [105]:
len(df_train.columns)

2263

In [106]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [107]:
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 2261) (61503, 2261)
[100]	training's auc: 0.81073	valid_1's auc: 0.778621
[200]	training's auc: 0.845424	valid_1's auc: 0.782825
[auc] tr:0.834352, va0.7839
-------------------- 1 --------------------
(246009, 2261) (61502, 2261)
[100]	training's auc: 0.8102	valid_1's auc: 0.781453
[200]	training's auc: 0.843997	valid_1's auc: 0.790121
[300]	training's auc: 0.869228	valid_1's auc: 0.791057
[auc] tr:0.859788, va0.7916
-------------------- 2 --------------------
(246009, 2261) (61502, 2261)
[100]	training's auc: 0.810551	valid_1's auc: 0.781575
[200]	training's auc: 0.844204	valid_1's auc: 0.785689
[300]	training's auc: 0.870153	valid_1's auc: 0.787473
[400]	training's auc: 0.890031	valid_1's auc: 0.787834
[auc] tr:0.878178, va0.7884
-------------------- 3 --------------------
(246009, 2261) (61502, 2261)
[100]	training's auc: 0.811071	valid_1's auc: 0.779379
[200]	training's auc: 0.845919	valid_1's auc: 0.78718
[300]	training's auc: 0

In [108]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
2253,app_NEW_EXT_SOURCES_MEAN,112020.506607,1144.874170
2228,app_AMT_CREDIT_PER_AMT_ANNUITY,7059.907736,469.018206
2238,app_DAYS_EMPLOYED_div_DAYS_BIRTH,4681.057034,461.122897
2229,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,4613.040843,329.057255
1379,DAYS_BIRTH,4279.070571,416.573247
2241,app_EXT_SOURCE_1_PER_DAYS_BIRTH,4203.736921,375.725360
2247,app_EXT_SOURCE_2_times_EXT_SOURCE_3,4026.441920,133.997063
383,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_D...,3953.013209,1006.864274
1722,PREV_NAME_CONTRACT_STATUS_Refused_MEAN,3496.569398,540.344102
1375,CODE_GENDER,3491.714947,235.759569


In [109]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [110]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [111]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_LgbFeatureSelection8.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.052987
1,100005,0.115211
2,100013,0.025382
3,100028,0.036537
4,100038,0.196259


### 変数選択009

In [112]:
len(df_train.columns)

2263

In [113]:
length = int(len(df_train.columns) * 0.95)
length

2149

In [114]:
cols = list(imp.sort_values("imp", ascending=False)['col'][:length])
if 'TARGET' not in cols:
    print('aa')
    
if 'SK_ID_CURR' not in cols:
    print('bb')
    
train_columns = cols.copy()
train_columns.append('SK_ID_CURR')
train_columns.append('TARGET')

test_columns = cols.copy()
test_columns.append('SK_ID_CURR')

aa
bb


In [115]:
del df_train
del df_test
gc.collect()
df_train = df_train_origin[train_columns].copy()
df_test = df_test_origin[test_columns].copy()
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 2522.38 MB
Memory usage of optimization is 2644902239.00 MB
Decreased by -104857500.0%
Memory usage of dataframe is 399.78 MB
Memory usage of optimization is 419198528.00 MB
Decreased by -104857500.0%


,app_NEW_EXT_SOURCES_MEAN,app_AMT_CREDIT_PER_AMT_ANNUITY,app_DAYS_EMPLOYED_div_DAYS_BIRTH,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,DAYS_BIRTH,app_EXT_SOURCE_1_PER_DAYS_BIRTH,app_EXT_SOURCE_2_times_EXT_SOURCE_3,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_DEBT_MIN,PREV_NAME_CONTRACT_STATUS_Refused_MEAN,CODE_GENDER,...,PREV_SELLERPLACE_AREA_SIZE,PREV_RATE_INTEREST_PRIMARY_SUM,PREV_RATE_INTEREST_PRIMARY_VAR,PREV_RATE_INTEREST_PRIVILEGED_MEAN,PREV_RATE_INTEREST_PRIVILEGED_MIN,PREV_RATE_INTEREST_PRIVILEGED_SIZE,PREV_RATE_INTEREST_PRIVILEGED_VAR,ACTIVE_DPD_4_cnt_SIZE_MAX,SK_ID_CURR,TARGET
0,-2.046875,-0.596191,-0.849609,0.388672,1.597656,0.141479,-1.687500,-0.399170,-0.683105,-1.387695,...,-1.130859,-0.123352,0.239136,0.27124,0.27124,-1.130859,0.239136,-1.062500,100002,1
1,-0.362793,1.686523,-0.817871,0.290283,-0.222778,-0.532715,1.546875,-0.328125,-0.683105,0.720215,...,-0.149780,-0.123352,0.239136,0.27124,0.27124,-0.149780,0.239136,0.765137,100003,0
2,0.889648,-0.109741,-1.407227,-1.044922,-0.716797,0.843262,0.272705,1.808594,-0.683105,-1.387695,...,-1.130859,-0.123352,0.239136,0.27124,0.27124,-1.130859,0.239136,0.765137,100004,0
3,0.950195,-1.558594,-0.159912,-0.514648,-0.708008,0.843262,1.546875,-0.403564,0.790039,0.720215,...,1.176758,-0.123352,0.239136,0.27124,0.27124,1.176758,0.239136,-1.665039,100006,0
4,-1.232422,0.326172,-0.206299,-1.044922,-0.901367,0.843262,1.546875,1.808594,-0.683105,-1.387695,...,0.656250,-0.123352,0.239136,0.27124,0.27124,0.656250,0.239136,0.765137,100007,0


In [116]:
len(df_train.columns)

2151

In [117]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [118]:
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 2149) (61503, 2149)
[100]	training's auc: 0.81094	valid_1's auc: 0.778719
[200]	training's auc: 0.844916	valid_1's auc: 0.786514
[300]	training's auc: 0.87063	valid_1's auc: 0.788002
[400]	training's auc: 0.890683	valid_1's auc: 0.788642
[500]	training's auc: 0.907568	valid_1's auc: 0.78905
[600]	training's auc: 0.920772	valid_1's auc: 0.788848
[auc] tr:0.906210, va0.7890
-------------------- 1 --------------------
(246009, 2149) (61502, 2149)
[100]	training's auc: 0.810677	valid_1's auc: 0.780744
[200]	training's auc: 0.845141	valid_1's auc: 0.789017
[300]	training's auc: 0.870097	valid_1's auc: 0.791226
[400]	training's auc: 0.889778	valid_1's auc: 0.791493
[500]	training's auc: 0.90667	valid_1's auc: 0.791428
[600]	training's auc: 0.920499	valid_1's auc: 0.791321
[auc] tr:0.908692, va0.7917
-------------------- 2 --------------------
(246009, 2149) (61502, 2149)
[100]	training's auc: 0.810436	valid_1's auc: 0.782295
[200]	training

In [119]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
2142,app_NEW_EXT_SOURCES_MEAN,112682.899443,1262.459096
2117,app_AMT_CREDIT_PER_AMT_ANNUITY,7492.676528,916.234402
2127,app_DAYS_EMPLOYED_div_DAYS_BIRTH,4735.331215,170.294815
2118,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,4714.559672,267.888601
1290,DAYS_BIRTH,4685.310134,432.643887
2130,app_EXT_SOURCE_1_PER_DAYS_BIRTH,4462.906354,41.886360
2136,app_EXT_SOURCE_2_times_EXT_SOURCE_3,4233.706144,369.427725
363,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_D...,3983.679467,893.959513
2116,app_AMT_CREDIT_AMT_GOODS_PRICE,3692.245592,421.528597
1371,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,3635.487297,296.284671


In [120]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [121]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [122]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_LgbFeatureSelection9.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.048708
1,100005,0.121432
2,100013,0.030778
3,100028,0.046486
4,100038,0.163791


### 変数選択009

In [123]:
len(df_train.columns)

2151

In [124]:
length = int(len(df_train.columns) * 0.95)
length

2043

In [125]:
cols = list(imp.sort_values("imp", ascending=False)['col'][:length])
if 'TARGET' not in cols:
    print('aa')
    
if 'SK_ID_CURR' not in cols:
    print('bb')
    
train_columns = cols.copy()
train_columns.append('SK_ID_CURR')
train_columns.append('TARGET')

test_columns = cols.copy()
test_columns.append('SK_ID_CURR')

aa
bb


In [126]:
del df_train
del df_test
gc.collect()
df_train = df_train_origin[train_columns].copy()
df_test = df_test_origin[test_columns].copy()
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 2398.03 MB
Memory usage of optimization is 2514517575.00 MB
Decreased by -104857500.0%
Memory usage of dataframe is 380.07 MB
Memory usage of optimization is 398531072.00 MB
Decreased by -104857500.0%


,app_NEW_EXT_SOURCES_MEAN,app_AMT_CREDIT_PER_AMT_ANNUITY,app_DAYS_EMPLOYED_div_DAYS_BIRTH,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,DAYS_BIRTH,app_EXT_SOURCE_1_PER_DAYS_BIRTH,app_EXT_SOURCE_2_times_EXT_SOURCE_3,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_DEBT_MIN,app_AMT_CREDIT_AMT_GOODS_PRICE,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,...,BURO_Non_zero_DPD_cnt_SIZE_VAR,BURO_SK_ID_BUREAU_MAX_MAX,ACTIVE_DPD_0_cnt_SIZE_SIZE,REFUSED_AMT_CREDIT_SIZE,ACTIVE_DPD_0_cnt_SIZE_MIN,ACTIVE_DPD_0_cnt_SIZE_MEAN,BURO_SK_ID_BUREAU_MAX_MEAN,REFUSED_AMT_DOWN_PAYMENT_SIZE,SK_ID_CURR,TARGET
0,-2.046875,-0.596191,-0.849609,0.388672,1.597656,0.141479,-1.687500,-0.399170,-0.032013,-1.705078,...,-1.208008,0.087097,-0.055267,0.778809,-1.273438,-1.144531,0.090576,0.778809,100002,1
1,-0.362793,1.686523,-0.817871,0.290283,-0.222778,-0.532715,1.546875,-0.328125,1.289062,-0.032410,...,0.784668,0.576660,-0.561523,0.778809,0.781738,0.776855,0.580078,0.778809,100003,0
2,0.889648,-0.109741,-1.407227,-1.044922,-0.716797,0.843262,0.272705,1.808594,-0.790039,-0.240479,...,0.784668,0.576660,1.738281,0.778809,0.781738,0.776855,0.580078,0.778809,100004,0
3,0.950195,-1.558594,-0.159912,-0.514648,-0.708008,0.843262,1.546875,-0.403564,-0.557129,-0.032410,...,-1.599609,-2.380859,-1.671875,-1.346680,-1.558594,-1.625000,-2.378906,-1.346680,100006,0
4,-1.232422,0.326172,-0.206299,-1.044922,-0.901367,0.843262,1.546875,1.808594,-0.790039,0.416260,...,0.784668,0.576660,1.738281,0.778809,0.781738,0.776855,0.580078,0.778809,100007,0


In [127]:
len(df_train.columns)

2045

In [128]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [129]:
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 2043) (61503, 2043)
[100]	training's auc: 0.81064	valid_1's auc: 0.77849
[200]	training's auc: 0.844839	valid_1's auc: 0.785934
[300]	training's auc: 0.87019	valid_1's auc: 0.78689
[400]	training's auc: 0.890924	valid_1's auc: 0.787442
[500]	training's auc: 0.907606	valid_1's auc: 0.787486
[auc] tr:0.893618, va0.7877
-------------------- 1 --------------------
(246009, 2043) (61502, 2043)
[100]	training's auc: 0.81054	valid_1's auc: 0.780958
[200]	training's auc: 0.844954	valid_1's auc: 0.789634
[300]	training's auc: 0.869757	valid_1's auc: 0.791189
[auc] tr:0.866258, va0.7913
-------------------- 2 --------------------
(246009, 2043) (61502, 2043)
[100]	training's auc: 0.810587	valid_1's auc: 0.782395
[200]	training's auc: 0.844433	valid_1's auc: 0.788997
[300]	training's auc: 0.87031	valid_1's auc: 0.790531
[400]	training's auc: 0.890345	valid_1's auc: 0.791178
[500]	training's auc: 0.90764	valid_1's auc: 0.79142
[600]	training's a

In [130]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
2036,app_NEW_EXT_SOURCES_MEAN,112623.698231,984.679271
2011,app_AMT_CREDIT_PER_AMT_ANNUITY,7511.636046,429.164427
2021,app_DAYS_EMPLOYED_div_DAYS_BIRTH,4781.190257,670.528967
2012,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,4634.576392,423.876988
1202,DAYS_BIRTH,4491.243669,265.563254
2024,app_EXT_SOURCE_1_PER_DAYS_BIRTH,4305.846792,516.832566
2030,app_EXT_SOURCE_2_times_EXT_SOURCE_3,4177.840982,357.443674
295,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_D...,3861.706289,996.105633
1283,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,3681.735385,412.457184
2010,app_AMT_CREDIT_AMT_GOODS_PRICE,3662.016300,401.239465


In [131]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [132]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [133]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_LgbFeatureSelection10.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.051351
1,100005,0.123765
2,100013,0.030665
3,100028,0.035735
4,100038,0.163064


### 変数選択011

In [134]:
len(df_train.columns)

2045

In [135]:
length = int(len(df_train.columns) * 0.95)
length

1942

In [136]:
cols = list(imp.sort_values("imp", ascending=False)['col'][:length])
if 'TARGET' not in cols:
    print('aa')
    
if 'SK_ID_CURR' not in cols:
    print('bb')
    
train_columns = cols.copy()
train_columns.append('SK_ID_CURR')
train_columns.append('TARGET')

test_columns = cols.copy()
test_columns.append('SK_ID_CURR')

aa
bb


In [137]:
del df_train
del df_test
gc.collect()
df_train = df_train_origin[train_columns].copy()
df_test = df_test_origin[test_columns].copy()
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 2279.55 MB
Memory usage of optimization is 2390283131.00 MB
Decreased by -104857500.0%
Memory usage of dataframe is 361.29 MB
Memory usage of optimization is 378838496.00 MB
Decreased by -104857500.0%


,app_NEW_EXT_SOURCES_MEAN,app_AMT_CREDIT_PER_AMT_ANNUITY,app_DAYS_EMPLOYED_div_DAYS_BIRTH,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,DAYS_BIRTH,app_EXT_SOURCE_1_PER_DAYS_BIRTH,app_EXT_SOURCE_2_times_EXT_SOURCE_3,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_DEBT_MIN,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,app_AMT_CREDIT_AMT_GOODS_PRICE,...,REFUSED_CHANNEL_TYPE_Channel_of_corporate_sales_MEAN,REFUSED_CHANNEL_TYPE_Car_dealer_MEAN,BURO_Last_status_4_MEAN_SIZE,BURO_When_closed_MIN_SIZE,REFUSED_AMT_GOODS_PRICE_SIZE,ACTIVE_DPD_1_Month_MAX_MIN,REFUSED_DAYS_FIRST_DRAWING_MAX,REFUSED_DAYS_FIRST_DRAWING_MIN,SK_ID_CURR,TARGET
0,-2.046875,-0.596191,-0.849609,0.388672,1.597656,0.141479,-1.687500,-0.399170,-1.705078,-0.032013,...,0.780762,0.782227,0.910156,0.910156,0.778809,-2.015625,0.0,0.0,100002,1
1,-0.362793,1.686523,-0.817871,0.290283,-0.222778,-0.532715,1.546875,-0.328125,-0.032410,1.289062,...,0.780762,0.782227,0.139893,0.139893,0.778809,0.493408,0.0,0.0,100003,0
2,0.889648,-0.109741,-1.407227,-1.044922,-0.716797,0.843262,0.272705,1.808594,-0.240479,-0.790039,...,0.780762,0.782227,-0.474365,-0.474365,0.778809,0.493408,0.0,0.0,100004,0
3,0.950195,-1.558594,-0.159912,-0.514648,-0.708008,0.843262,1.546875,-0.403564,-0.032410,-0.557129,...,-1.284180,-1.278320,-1.640625,-1.640625,-1.346680,-2.029297,0.0,0.0,100006,0
4,-1.232422,0.326172,-0.206299,-1.044922,-0.901367,0.843262,1.546875,1.808594,0.416260,-0.790039,...,0.780762,0.782227,-0.928223,-0.928223,0.778809,0.493408,0.0,0.0,100007,0


In [138]:
len(df_train.columns)

1944

In [139]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [140]:
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 1942) (61503, 1942)
[100]	training's auc: 0.811239	valid_1's auc: 0.778998
[200]	training's auc: 0.845043	valid_1's auc: 0.785824
[300]	training's auc: 0.870187	valid_1's auc: 0.787418
[400]	training's auc: 0.890069	valid_1's auc: 0.78768
[500]	training's auc: 0.906545	valid_1's auc: 0.787505
[auc] tr:0.890328, va0.7879
-------------------- 1 --------------------
(246009, 1942) (61502, 1942)
[100]	training's auc: 0.810875	valid_1's auc: 0.781185
[200]	training's auc: 0.844584	valid_1's auc: 0.78947
[300]	training's auc: 0.870404	valid_1's auc: 0.790781
[400]	training's auc: 0.890898	valid_1's auc: 0.791061
[auc] tr:0.882564, va0.7913
-------------------- 2 --------------------
(246009, 1942) (61502, 1942)
[100]	training's auc: 0.810437	valid_1's auc: 0.782097
[200]	training's auc: 0.844304	valid_1's auc: 0.788609
[300]	training's auc: 0.869506	valid_1's auc: 0.790026
[400]	training's auc: 0.889776	valid_1's auc: 0.7908
[500]	training

In [141]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
1935,app_NEW_EXT_SOURCES_MEAN,112565.831048,1070.142375
1910,app_AMT_CREDIT_PER_AMT_ANNUITY,7440.793497,499.720179
1911,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,4847.553265,312.855734
1920,app_DAYS_EMPLOYED_div_DAYS_BIRTH,4830.541175,565.262033
1140,DAYS_BIRTH,4750.456211,408.151463
1923,app_EXT_SOURCE_1_PER_DAYS_BIRTH,4399.244352,320.714399
1929,app_EXT_SOURCE_2_times_EXT_SOURCE_3,4337.824089,340.812406
256,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_D...,3870.196958,952.121595
1221,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,3680.869448,399.873973
1909,app_AMT_CREDIT_AMT_GOODS_PRICE,3607.921303,347.510911


In [142]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [143]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [144]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_LgbFeatureSelection11.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.050674
1,100005,0.111354
2,100013,0.025098
3,100028,0.032931
4,100038,0.154721


### 変数選択012

In [145]:
len(df_train.columns)

1944

In [146]:
length = int(len(df_train.columns) * 0.95)
length

1846

In [147]:
cols = list(imp.sort_values("imp", ascending=False)['col'][:length])
if 'TARGET' not in cols:
    print('aa')
    
if 'SK_ID_CURR' not in cols:
    print('bb')
    
train_columns = cols.copy()
train_columns.append('SK_ID_CURR')
train_columns.append('TARGET')

test_columns = cols.copy()
test_columns.append('SK_ID_CURR')

aa
bb


In [148]:
del df_train
del df_test
gc.collect()
df_train = df_train_origin[train_columns].copy()
df_test = df_test_origin[test_columns].copy()
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 2166.94 MB
Memory usage of optimization is 2272198907.00 MB
Decreased by -104857500.0%
Memory usage of dataframe is 343.44 MB
Memory usage of optimization is 360120800.00 MB
Decreased by -104857500.0%


,app_NEW_EXT_SOURCES_MEAN,app_AMT_CREDIT_PER_AMT_ANNUITY,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,app_DAYS_EMPLOYED_div_DAYS_BIRTH,DAYS_BIRTH,app_EXT_SOURCE_1_PER_DAYS_BIRTH,app_EXT_SOURCE_2_times_EXT_SOURCE_3,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_DEBT_MIN,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,app_AMT_CREDIT_AMT_GOODS_PRICE,...,BURO_STATUS_5_MEAN_SIZE,BURO_Non_zero_DPD_cnt_SIZE_VAR,REFUSED_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_MEAN,REFUSED_NAME_CONTRACT_STATUS_Refused_MEAN,REFUSED_NAME_CONTRACT_STATUS_Canceled_MEAN,REFUSED_NAME_CONTRACT_STATUS_Approved_MEAN,REFUSED_NAME_CLIENT_TYPE_XNA_MEAN,BURO_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_DEBT_SIZE,SK_ID_CURR,TARGET
0,-2.046875,-0.596191,0.388672,-0.849609,1.597656,0.141479,-1.687500,-0.399170,-1.705078,-0.032013,...,0.910156,-1.208008,0.782227,0.763672,0.0,0.0,0.782227,0.910156,100002,1
1,-0.362793,1.686523,0.290283,-0.817871,-0.222778,-0.532715,1.546875,-0.328125,-0.032410,1.289062,...,0.139893,0.784668,0.782227,0.763672,0.0,0.0,0.782227,0.139893,100003,0
2,0.889648,-0.109741,-1.044922,-1.407227,-0.716797,0.843262,0.272705,1.808594,-0.240479,-0.790039,...,-0.474365,0.784668,0.782227,0.763672,0.0,0.0,0.782227,-0.474365,100004,0
3,0.950195,-1.558594,-0.514648,-0.159912,-0.708008,0.843262,1.546875,-0.403564,-0.032410,-0.557129,...,-1.640625,-1.599609,-1.278320,-1.106445,0.0,0.0,-1.279297,-1.640625,100006,0
4,-1.232422,0.326172,-1.044922,-0.206299,-0.901367,0.843262,1.546875,1.808594,0.416260,-0.790039,...,-0.928223,0.784668,0.782227,0.763672,0.0,0.0,0.782227,-0.928223,100007,0


In [149]:
len(df_train.columns)

1848

In [150]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [151]:
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 1846) (61503, 1846)
[100]	training's auc: 0.81056	valid_1's auc: 0.777934
[200]	training's auc: 0.84492	valid_1's auc: 0.785734
[300]	training's auc: 0.870552	valid_1's auc: 0.787015
[400]	training's auc: 0.891052	valid_1's auc: 0.78762
[auc] tr:0.886149, va0.7878
-------------------- 1 --------------------
(246009, 1846) (61502, 1846)
[100]	training's auc: 0.810392	valid_1's auc: 0.78146
[200]	training's auc: 0.844277	valid_1's auc: 0.789859
[300]	training's auc: 0.869779	valid_1's auc: 0.79146
[400]	training's auc: 0.889947	valid_1's auc: 0.791869
[500]	training's auc: 0.906791	valid_1's auc: 0.792417
[600]	training's auc: 0.920793	valid_1's auc: 0.791921
[auc] tr:0.907285, va0.7925
-------------------- 2 --------------------
(246009, 1846) (61502, 1846)
[100]	training's auc: 0.810404	valid_1's auc: 0.782348
[200]	training's auc: 0.844022	valid_1's auc: 0.789706
[300]	training's auc: 0.869476	valid_1's auc: 0.791074
[400]	training'

In [152]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
1839,app_NEW_EXT_SOURCES_MEAN,112683.104552,1245.577367
1814,app_AMT_CREDIT_PER_AMT_ANNUITY,7430.307892,581.434452
1824,app_DAYS_EMPLOYED_div_DAYS_BIRTH,4912.330643,451.611087
1098,DAYS_BIRTH,4832.751315,508.665330
1815,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,4807.859635,364.553886
1827,app_EXT_SOURCE_1_PER_DAYS_BIRTH,4326.383385,276.464970
1833,app_EXT_SOURCE_2_times_EXT_SOURCE_3,4208.369443,254.473137
246,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_D...,3889.278417,1023.048814
1094,CODE_GENDER,3571.324077,200.304969
1179,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,3545.086981,372.416963


In [153]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [154]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [155]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_LgbFeatureSelection12.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.047885
1,100005,0.110154
2,100013,0.026342
3,100028,0.038437
4,100038,0.164935


### 変数選択013

In [156]:
len(df_train.columns)

1848

In [157]:
length = int(len(df_train.columns) * 0.95)
length

1755

In [158]:
cols = list(imp.sort_values("imp", ascending=False)['col'][:length])
if 'TARGET' not in cols:
    print('aa')
    
if 'SK_ID_CURR' not in cols:
    print('bb')
    
train_columns = cols.copy()
train_columns.append('SK_ID_CURR')
train_columns.append('TARGET')

test_columns = cols.copy()
test_columns.append('SK_ID_CURR')

aa
bb


In [159]:
del df_train
del df_test
gc.collect()
df_train = df_train_origin[train_columns].copy()
df_test = df_test_origin[test_columns].copy()
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 2060.19 MB
Memory usage of optimization is 2160264903.00 MB
Decreased by -104857500.0%
Memory usage of dataframe is 326.52 MB
Memory usage of optimization is 342377984.00 MB
Decreased by -104857500.0%


,app_NEW_EXT_SOURCES_MEAN,app_AMT_CREDIT_PER_AMT_ANNUITY,app_DAYS_EMPLOYED_div_DAYS_BIRTH,DAYS_BIRTH,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,app_EXT_SOURCE_1_PER_DAYS_BIRTH,app_EXT_SOURCE_2_times_EXT_SOURCE_3,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_DEBT_MIN,CODE_GENDER,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,...,ACTIVE_CREDIT_TYPE_Loan_for_business_development_MEAN,BURO_DPD_5_cnt_MAX_MEAN,BURO_First_status_4_MEAN_MAX,ACTIVE_CREDIT_TYPE_Loan_for_purchase_of_shares__margin_lending__MEAN,BURO_DPD_5_cnt_MAX_SUM,BURO_First_status_3_MEAN_VAR,ACTIVE_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_MEAN,BURO_DPD_5_cnt_MIN_MAX,SK_ID_CURR,TARGET
0,-2.046875,-0.596191,-0.849609,1.597656,0.388672,0.141479,-1.687500,-0.399170,-1.387695,-1.705078,...,-0.422607,0.420410,-1.121094,-0.421143,-0.085022,0.0,-0.421143,0.420410,100002,1
1,-0.362793,1.686523,-0.817871,-0.222778,0.290283,-0.532715,1.546875,-0.328125,0.720215,-0.032410,...,-0.422607,0.420410,0.892090,-0.421143,-0.085022,0.0,-0.421143,0.420410,100003,0
2,0.889648,-0.109741,-1.407227,-0.716797,-1.044922,0.843262,0.272705,1.808594,-1.387695,-0.240479,...,2.367188,0.420410,0.892090,2.373047,-0.085022,0.0,2.373047,0.420410,100004,0
3,0.950195,-1.558594,-0.159912,-0.708008,-0.514648,0.843262,1.546875,-0.403564,0.720215,-0.032410,...,-0.422607,-2.382812,-1.121094,-0.421143,-0.085022,0.0,-0.421143,-2.384766,100006,0
4,-1.232422,0.326172,-0.206299,-0.901367,-1.044922,0.843262,1.546875,1.808594,-1.387695,0.416260,...,2.367188,0.420410,0.892090,2.373047,-0.085022,0.0,2.373047,0.420410,100007,0


In [160]:
len(df_train.columns)

1757

In [161]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [162]:
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 1755) (61503, 1755)
[100]	training's auc: 0.810626	valid_1's auc: 0.778723
[200]	training's auc: 0.844583	valid_1's auc: 0.78617
[300]	training's auc: 0.870712	valid_1's auc: 0.786899
[400]	training's auc: 0.89104	valid_1's auc: 0.787512
[500]	training's auc: 0.908284	valid_1's auc: 0.787891
[600]	training's auc: 0.92191	valid_1's auc: 0.788037
[auc] tr:0.908464, va0.7881
-------------------- 1 --------------------
(246009, 1755) (61502, 1755)
[100]	training's auc: 0.810643	valid_1's auc: 0.780904
[200]	training's auc: 0.845064	valid_1's auc: 0.789818
[300]	training's auc: 0.870453	valid_1's auc: 0.791706
[400]	training's auc: 0.890615	valid_1's auc: 0.79167
[auc] tr:0.882746, va0.7921
-------------------- 2 --------------------
(246009, 1755) (61502, 1755)
[100]	training's auc: 0.81077	valid_1's auc: 0.782577
[200]	training's auc: 0.844227	valid_1's auc: 0.789438
[300]	training's auc: 0.869718	valid_1's auc: 0.790532
[400]	training'

In [163]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
1748,app_NEW_EXT_SOURCES_MEAN,112885.263211,884.415420
1723,app_AMT_CREDIT_PER_AMT_ANNUITY,7578.748626,370.164943
1733,app_DAYS_EMPLOYED_div_DAYS_BIRTH,4959.053815,600.199735
1038,DAYS_BIRTH,4784.646671,243.013630
1724,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,4736.666330,377.782723
1736,app_EXT_SOURCE_1_PER_DAYS_BIRTH,4409.421458,295.197516
1742,app_EXT_SOURCE_2_times_EXT_SOURCE_3,4252.794545,277.254579
220,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_D...,4078.339364,944.533376
1119,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,3726.142515,407.762459
1722,app_AMT_CREDIT_AMT_GOODS_PRICE,3714.349835,385.757262


In [164]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [165]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [166]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_LgbFeatureSelection13.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.048524
1,100005,0.129395
2,100013,0.028917
3,100028,0.034393
4,100038,0.165921


### 変数選択014

In [167]:
len(df_train.columns)

1757

In [168]:
length = int(len(df_train.columns) * 0.90)
length

1581

In [169]:
cols = list(imp.sort_values("imp", ascending=False)['col'][:length])
if 'TARGET' not in cols:
    print('aa')
    
if 'SK_ID_CURR' not in cols:
    print('bb')
    
train_columns = cols.copy()
train_columns.append('SK_ID_CURR')
train_columns.append('TARGET')

test_columns = cols.copy()
test_columns.append('SK_ID_CURR')

aa
bb


In [170]:
del df_train
del df_test
gc.collect()
df_train = df_train_origin[train_columns].copy()
df_test = df_test_origin[test_columns].copy()
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 1856.08 MB
Memory usage of optimization is 1946237247.00 MB
Decreased by -104857500.0%
Memory usage of dataframe is 294.16 MB
Memory usage of optimization is 308452160.00 MB
Decreased by -104857500.0%


,app_NEW_EXT_SOURCES_MEAN,app_AMT_CREDIT_PER_AMT_ANNUITY,app_DAYS_EMPLOYED_div_DAYS_BIRTH,DAYS_BIRTH,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,app_EXT_SOURCE_1_PER_DAYS_BIRTH,app_EXT_SOURCE_2_times_EXT_SOURCE_3,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_DEBT_MIN,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,app_AMT_CREDIT_AMT_GOODS_PRICE,...,BURO_STATUS_3_MEAN_MEAN,REFUSED_prev_DAYS_TERMINATION_less__500_SIZE,REFUSED_prev_DAYS_TERMINATION_less__500_MIN,REFUSED_prev_missing_MAX,REFUSED_prev_DAYS_TERMINATION_less__500_SUM,ACTIVE_AMT_CREDIT_SUM_LIMIT_SIZE,ACTIVE_First_status_C_MEAN_MEAN,ACTIVE_AMT_CREDIT_SUM_SIZE,SK_ID_CURR,TARGET
0,-2.046875,-0.596191,-0.849609,1.597656,0.388672,0.141479,-1.687500,-0.399170,-1.705078,-0.032013,...,-1.12207,0.778809,0.0,0.752930,0.0,-0.055267,-1.249023,-0.055267,100002,1
1,-0.362793,1.686523,-0.817871,-0.222778,0.290283,-0.532715,1.546875,-0.328125,-0.032410,1.289062,...,0.89209,0.778809,0.0,0.752930,0.0,-0.561523,0.802246,-0.561523,100003,0
2,0.889648,-0.109741,-1.407227,-0.716797,-1.044922,0.843262,0.272705,1.808594,-0.240479,-0.790039,...,0.89209,0.778809,0.0,0.752930,0.0,1.738281,0.802246,1.738281,100004,0
3,0.950195,-1.558594,-0.159912,-0.708008,-0.514648,0.843262,1.546875,-0.403564,-0.032410,-0.557129,...,-1.12207,-1.346680,0.0,-1.089844,0.0,-1.671875,-1.249023,-1.671875,100006,0
4,-1.232422,0.326172,-0.206299,-0.901367,-1.044922,0.843262,1.546875,1.808594,0.416260,-0.790039,...,0.89209,0.778809,0.0,0.752930,0.0,1.738281,0.802246,1.738281,100007,0


In [171]:
len(df_train.columns)

1583

In [172]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [173]:
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 1581) (61503, 1581)
[100]	training's auc: 0.811126	valid_1's auc: 0.777896
[200]	training's auc: 0.845493	valid_1's auc: 0.784943
[300]	training's auc: 0.870505	valid_1's auc: 0.785526
[400]	training's auc: 0.890474	valid_1's auc: 0.785515
[auc] tr:0.872792, va0.7857
-------------------- 1 --------------------
(246009, 1581) (61502, 1581)
[100]	training's auc: 0.809942	valid_1's auc: 0.781245
[200]	training's auc: 0.844685	valid_1's auc: 0.789683
[300]	training's auc: 0.870152	valid_1's auc: 0.790982
[400]	training's auc: 0.890167	valid_1's auc: 0.791138
[500]	training's auc: 0.907254	valid_1's auc: 0.791057
[auc] tr:0.894327, va0.7912
-------------------- 2 --------------------
(246009, 1581) (61502, 1581)
[100]	training's auc: 0.810463	valid_1's auc: 0.782413
[200]	training's auc: 0.844231	valid_1's auc: 0.789646
[300]	training's auc: 0.8692	valid_1's auc: 0.791363
[400]	training's auc: 0.889014	valid_1's auc: 0.791953
[500]	traini

In [174]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
1574,app_NEW_EXT_SOURCES_MEAN,112958.796957,1233.895648
1550,app_AMT_CREDIT_PER_AMT_ANNUITY,7634.537381,783.509990
1560,app_DAYS_EMPLOYED_div_DAYS_BIRTH,5040.596843,667.365968
918,DAYS_BIRTH,4818.774355,575.137984
1551,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,4778.946915,242.466973
1569,app_EXT_SOURCE_2_times_EXT_SOURCE_3,4431.801769,136.093304
1563,app_EXT_SOURCE_1_PER_DAYS_BIRTH,4345.462940,294.825625
163,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_D...,4062.137785,1003.678504
999,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,3745.916153,349.481790
1549,app_AMT_CREDIT_AMT_GOODS_PRICE,3668.506573,598.440034


In [175]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [176]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [177]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_LgbFeatureSelection14.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.056879
1,100005,0.102419
2,100013,0.025238
3,100028,0.040458
4,100038,0.149693


### 変数選択015

In [178]:
len(df_train.columns)

1583

In [179]:
length = int(len(df_train.columns) * 0.95)
length

1503

In [180]:
cols = list(imp.sort_values("imp", ascending=False)['col'][:length])
if 'TARGET' not in cols:
    print('aa')
    
if 'SK_ID_CURR' not in cols:
    print('bb')
    
train_columns = cols.copy()
train_columns.append('SK_ID_CURR')
train_columns.append('TARGET')

test_columns = cols.copy()
test_columns.append('SK_ID_CURR')

aa
bb


In [181]:
del df_train
del df_test
gc.collect()
df_train = df_train_origin[train_columns].copy()
df_test = df_test_origin[test_columns].copy()
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 1764.58 MB
Memory usage of optimization is 1850293815.00 MB
Decreased by -104857500.0%
Memory usage of dataframe is 279.66 MB
Memory usage of optimization is 293244032.00 MB
Decreased by -104857500.0%


,app_NEW_EXT_SOURCES_MEAN,app_AMT_CREDIT_PER_AMT_ANNUITY,app_DAYS_EMPLOYED_div_DAYS_BIRTH,DAYS_BIRTH,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,app_EXT_SOURCE_2_times_EXT_SOURCE_3,app_EXT_SOURCE_1_PER_DAYS_BIRTH,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_DEBT_MIN,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,app_AMT_CREDIT_AMT_GOODS_PRICE,...,APPROVED_NAME_CASH_LOAN_PURPOSE_Car_repairs_MEAN,CLOSED_STATUS_1_MEAN_MIN,PREV_FLAG_LAST_APPL_PER_CONTRACT_N_MEAN,REFUSED_NAME_GOODS_CATEGORY_Computers_MEAN,INS_ins_DAYS_INSTALMENT_more_NUM_INSTALMENT_NUMBER_VAR,ORGANIZATION_TYPE_Insurance,APPROVED_NFLAG_INSURED_ON_APPROVAL_SUM,NAME_INCOME_TYPE_Commercial_associate,SK_ID_CURR,TARGET
0,-2.046875,-0.596191,-0.849609,1.597656,0.388672,-1.687500,0.141479,-0.399170,-1.705078,-0.032013,...,-0.06488,-1.214844,-0.128906,0.771973,-0.102051,-0.044098,-0.959473,-0.550781,100002,1
1,-0.362793,1.686523,-0.817871,-0.222778,0.290283,1.546875,-0.532715,-0.328125,-0.032410,1.289062,...,-0.06488,0.826172,-0.128906,0.771973,-0.102051,-0.044098,1.166992,-0.550781,100003,0
2,0.889648,-0.109741,-1.407227,-0.716797,-1.044922,0.272705,0.843262,1.808594,-0.240479,-0.790039,...,-0.06488,0.826172,-0.128906,0.771973,-0.102051,-0.044098,-0.959473,-0.550781,100004,0
3,0.950195,-1.558594,-0.159912,-0.708008,-0.514648,1.546875,0.843262,-0.403564,-0.032410,-0.557129,...,-0.06488,-1.214844,-0.128906,-1.324219,-0.102051,-0.044098,-0.959473,-0.550781,100006,0
4,-1.232422,0.326172,-0.206299,-0.901367,-1.044922,1.546875,0.843262,1.808594,0.416260,-0.790039,...,-0.06488,0.826172,-0.128906,0.771973,-0.102051,-0.044098,1.455078,-0.550781,100007,0


In [182]:
len(df_train.columns)

1505

In [183]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [184]:
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 1503) (61503, 1503)
[100]	training's auc: 0.810576	valid_1's auc: 0.77816
[200]	training's auc: 0.845077	valid_1's auc: 0.78532
[300]	training's auc: 0.870083	valid_1's auc: 0.786246
[400]	training's auc: 0.890316	valid_1's auc: 0.786284
[500]	training's auc: 0.906869	valid_1's auc: 0.786521
[auc] tr:0.904142, va0.7868
-------------------- 1 --------------------
(246009, 1503) (61502, 1503)
[100]	training's auc: 0.8107	valid_1's auc: 0.780573
[200]	training's auc: 0.844767	valid_1's auc: 0.789087
[300]	training's auc: 0.870778	valid_1's auc: 0.790937
[400]	training's auc: 0.890902	valid_1's auc: 0.791397
[500]	training's auc: 0.907246	valid_1's auc: 0.791627
[600]	training's auc: 0.921132	valid_1's auc: 0.792012
[700]	training's auc: 0.93309	valid_1's auc: 0.791754
[auc] tr:0.922767, va0.7922
-------------------- 2 --------------------
(246009, 1503) (61502, 1503)
[100]	training's auc: 0.810264	valid_1's auc: 0.781661
[200]	training'

In [185]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
1496,app_NEW_EXT_SOURCES_MEAN,112937.047807,1283.637208
1472,app_AMT_CREDIT_PER_AMT_ANNUITY,7634.618386,936.896535
1482,app_DAYS_EMPLOYED_div_DAYS_BIRTH,4921.465687,369.283818
1473,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,4919.817090,204.071506
872,DAYS_BIRTH,4650.516525,425.352214
1485,app_EXT_SOURCE_1_PER_DAYS_BIRTH,4500.766024,205.663291
1491,app_EXT_SOURCE_2_times_EXT_SOURCE_3,4299.917683,339.951306
141,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_D...,4061.482078,935.799248
953,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,3776.085937,336.475582
1471,app_AMT_CREDIT_AMT_GOODS_PRICE,3609.013968,420.349163


In [186]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [187]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [188]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_LgbFeatureSelection15.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.049797
1,100005,0.115536
2,100013,0.034282
3,100028,0.036639
4,100038,0.159417


### 変数選択016

In [189]:
len(df_train.columns)

1505

In [190]:
length = int(len(df_train.columns) * 0.95)
length

1429

In [191]:
cols = list(imp.sort_values("imp", ascending=False)['col'][:length])
if 'TARGET' not in cols:
    print('aa')
    
if 'SK_ID_CURR' not in cols:
    print('bb')
    
train_columns = cols.copy()
train_columns.append('SK_ID_CURR')
train_columns.append('TARGET')

test_columns = cols.copy()
test_columns.append('SK_ID_CURR')

aa
bb


In [192]:
del df_train
del df_test
gc.collect()
df_train = df_train_origin[train_columns].copy()
df_test = df_test_origin[test_columns].copy()
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 1677.77 MB
Memory usage of optimization is 1759270559.00 MB
Decreased by -104857500.0%
Memory usage of dataframe is 265.90 MB
Memory usage of optimization is 278815808.00 MB
Decreased by -104857500.0%


,app_NEW_EXT_SOURCES_MEAN,app_AMT_CREDIT_PER_AMT_ANNUITY,app_DAYS_EMPLOYED_div_DAYS_BIRTH,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,DAYS_BIRTH,app_EXT_SOURCE_1_PER_DAYS_BIRTH,app_EXT_SOURCE_2_times_EXT_SOURCE_3,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_DEBT_MIN,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,app_AMT_CREDIT_AMT_GOODS_PRICE,...,PREV_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_MEAN,REFUSED_PRODUCT_COMBINATION_Cash_Street__high_MEAN,NAME_EDUCATION_TYPE_Academic_degree,FLAG_DOCUMENT_5,PREV_NFLAG_LAST_APPL_IN_DAY_SUM,APPROVED_PRODUCT_COMBINATION_POS_other_with_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Cash_MEAN,CARD_MONTHS_BALANCE_SUM,SK_ID_CURR,TARGET
0,-2.046875,-0.596191,-0.849609,0.388672,1.597656,0.141479,-1.687500,-0.399170,-1.705078,-0.032013,...,-0.075195,0.771973,-0.023102,-0.123901,-1.129883,4.203125,0.770508,0.623047,100002,1
1,-0.362793,1.686523,-0.817871,0.290283,-0.222778,-0.532715,1.546875,-0.328125,-0.032410,1.289062,...,-0.075195,0.771973,-0.023102,-0.123901,-0.147095,-0.238037,0.770508,0.623047,100003,0
2,0.889648,-0.109741,-1.407227,-1.044922,-0.716797,0.843262,0.272705,1.808594,-0.240479,-0.790039,...,-0.075195,0.771973,-0.023102,-0.123901,-1.129883,-0.238037,0.770508,0.623047,100004,0
3,0.950195,-1.558594,-0.159912,-0.514648,-0.708008,0.843262,1.546875,-0.403564,-0.032410,-0.557129,...,-0.075195,-1.333008,-0.023102,-0.123901,1.181641,-0.238037,-1.340820,-1.296875,100006,0
4,-1.232422,0.326172,-0.206299,-1.044922,-0.901367,0.843262,1.546875,1.808594,0.416260,-0.790039,...,-0.075195,0.771973,-0.023102,-0.123901,0.660156,-0.238037,0.770508,0.623047,100007,0


In [193]:
len(df_train.columns)

1431

In [194]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [195]:
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 1429) (61503, 1429)
[100]	training's auc: 0.811236	valid_1's auc: 0.778639
[200]	training's auc: 0.84513	valid_1's auc: 0.785771
[300]	training's auc: 0.870339	valid_1's auc: 0.786729
[400]	training's auc: 0.890529	valid_1's auc: 0.787486
[auc] tr:0.888845, va0.7876
-------------------- 1 --------------------
(246009, 1429) (61502, 1429)
[100]	training's auc: 0.810749	valid_1's auc: 0.780891
[200]	training's auc: 0.844748	valid_1's auc: 0.789195
[300]	training's auc: 0.869719	valid_1's auc: 0.790542
[400]	training's auc: 0.890448	valid_1's auc: 0.790765
[500]	training's auc: 0.906934	valid_1's auc: 0.79118
[auc] tr:0.905383, va0.7912
-------------------- 2 --------------------
(246009, 1429) (61502, 1429)
[100]	training's auc: 0.810853	valid_1's auc: 0.783133
[200]	training's auc: 0.844558	valid_1's auc: 0.789695
[300]	training's auc: 0.869548	valid_1's auc: 0.791132
[400]	training's auc: 0.889763	valid_1's auc: 0.791344
[500]	traini

In [196]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
1422,app_NEW_EXT_SOURCES_MEAN,113005.745068,1065.214452
1398,app_AMT_CREDIT_PER_AMT_ANNUITY,7383.551766,619.844828
1408,app_DAYS_EMPLOYED_div_DAYS_BIRTH,4904.912721,336.443483
1399,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,4892.511949,259.267113
829,DAYS_BIRTH,4551.680619,417.217761
1411,app_EXT_SOURCE_1_PER_DAYS_BIRTH,4434.693282,189.239023
1417,app_EXT_SOURCE_2_times_EXT_SOURCE_3,4150.209293,223.768918
129,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_D...,3832.423555,1017.958972
908,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,3621.819711,444.275532
1397,app_AMT_CREDIT_AMT_GOODS_PRICE,3548.773837,384.054590


In [197]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [198]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [199]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_LgbFeatureSelection16.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.062864
1,100005,0.114234
2,100013,0.030747
3,100028,0.036082
4,100038,0.159958


### 変数選択017

In [200]:
len(df_train.columns)

1431

In [201]:
length = int(len(df_train.columns) * 0.95)
length

1359

In [202]:
cols = list(imp.sort_values("imp", ascending=False)['col'][:length])
if 'TARGET' not in cols:
    print('aa')
    
if 'SK_ID_CURR' not in cols:
    print('bb')
    
train_columns = cols.copy()
train_columns.append('SK_ID_CURR')
train_columns.append('TARGET')

test_columns = cols.copy()
test_columns.append('SK_ID_CURR')

aa
bb


In [203]:
del df_train
del df_test
gc.collect()
df_train = df_train_origin[train_columns].copy()
df_test = df_test_origin[test_columns].copy()
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 1595.66 MB
Memory usage of optimization is 1673167479.00 MB
Decreased by -104857500.0%
Memory usage of dataframe is 252.88 MB
Memory usage of optimization is 265167488.00 MB
Decreased by -104857500.0%


,app_NEW_EXT_SOURCES_MEAN,app_AMT_CREDIT_PER_AMT_ANNUITY,app_DAYS_EMPLOYED_div_DAYS_BIRTH,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,DAYS_BIRTH,app_EXT_SOURCE_1_PER_DAYS_BIRTH,app_EXT_SOURCE_2_times_EXT_SOURCE_3,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_DEBT_MIN,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,app_AMT_CREDIT_AMT_GOODS_PRICE,...,ORGANIZATION_TYPE_Transport__type_4,CLOSED_STATUS_1_MEAN_MEAN,REFUSED_NAME_SELLER_INDUSTRY_Consumer_electronics_MEAN,APPROVED_NAME_CASH_LOAN_PURPOSE_Medicine_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,ORGANIZATION_TYPE_Restaurant,CLOSED_STATUS_X_MEAN_MIN,CARD_AMT_RECIVABLE_SUM,SK_ID_CURR,TARGET
0,-2.046875,-0.596191,-0.849609,0.388672,1.597656,0.141479,-1.687500,-0.399170,-1.705078,-0.032013,...,-0.133667,-0.837891,0.752441,-0.077759,-0.11377,-0.076965,-1.260742,-0.258545,100002,1
1,-0.362793,1.686523,-0.817871,0.290283,-0.222778,-0.532715,1.546875,-0.328125,-0.032410,1.289062,...,-0.133667,0.825684,0.752441,-0.077759,-0.11377,-0.076965,0.817383,-0.258545,100003,0
2,0.889648,-0.109741,-1.407227,-1.044922,-0.716797,0.843262,0.272705,1.808594,-0.240479,-0.790039,...,-0.133667,0.825684,0.752441,-0.077759,-0.11377,-0.076965,0.817383,-0.258545,100004,0
3,0.950195,-1.558594,-0.159912,-0.514648,-0.708008,0.843262,1.546875,-0.403564,-0.032410,-0.557129,...,-0.133667,-1.220703,-1.440430,-0.077759,-0.11377,-0.076965,-1.260742,-0.258545,100006,0
4,-1.232422,0.326172,-0.206299,-1.044922,-0.901367,0.843262,1.546875,1.808594,0.416260,-0.790039,...,-0.133667,0.825684,0.752441,-0.077759,-0.11377,-0.076965,0.817383,-0.258545,100007,0


In [204]:
len(df_train.columns)

1361

In [205]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [206]:
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 1359) (61503, 1359)
[100]	training's auc: 0.81075	valid_1's auc: 0.778531
[200]	training's auc: 0.844744	valid_1's auc: 0.786452
[300]	training's auc: 0.869918	valid_1's auc: 0.787746
[auc] tr:0.861415, va0.7879
-------------------- 1 --------------------
(246009, 1359) (61502, 1359)
[100]	training's auc: 0.810974	valid_1's auc: 0.781165
[200]	training's auc: 0.845185	valid_1's auc: 0.78998
[300]	training's auc: 0.870089	valid_1's auc: 0.791948
[400]	training's auc: 0.890826	valid_1's auc: 0.792491
[500]	training's auc: 0.907418	valid_1's auc: 0.792847
[600]	training's auc: 0.921651	valid_1's auc: 0.792981
[700]	training's auc: 0.932824	valid_1's auc: 0.792959
[auc] tr:0.921784, va0.7931
-------------------- 2 --------------------
(246009, 1359) (61502, 1359)
[100]	training's auc: 0.81089	valid_1's auc: 0.782682
[200]	training's auc: 0.844303	valid_1's auc: 0.78932
[300]	training's auc: 0.869218	valid_1's auc: 0.790597
[400]	training

In [207]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
1352,app_NEW_EXT_SOURCES_MEAN,112610.341055,1162.302602
1328,app_AMT_CREDIT_PER_AMT_ANNUITY,7293.540553,508.473950
1329,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,4879.978229,343.785951
1338,app_DAYS_EMPLOYED_div_DAYS_BIRTH,4867.154303,282.915876
792,DAYS_BIRTH,4715.655567,699.305547
1341,app_EXT_SOURCE_1_PER_DAYS_BIRTH,4463.146068,307.296747
1347,app_EXT_SOURCE_2_times_EXT_SOURCE_3,4229.089985,414.664052
122,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_D...,4021.867539,1034.640255
865,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,3754.891745,486.617227
788,CODE_GENDER,3513.559208,223.638503


In [208]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [209]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [210]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_LgbFeatureSelection17.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.056230
1,100005,0.118310
2,100013,0.029540
3,100028,0.040367
4,100038,0.152151


### 変数選択018

In [211]:
len(df_train.columns)

1361

In [212]:
length = int(len(df_train.columns) * 0.90)
length

1224

In [213]:
cols = list(imp.sort_values("imp", ascending=False)['col'][:length])
if 'TARGET' not in cols:
    print('aa')
    
if 'SK_ID_CURR' not in cols:
    print('bb')
    
train_columns = cols.copy()
train_columns.append('SK_ID_CURR')
train_columns.append('TARGET')

test_columns = cols.copy()
test_columns.append('SK_ID_CURR')

aa
bb


In [214]:
del df_train
del df_test
gc.collect()
df_train = df_train_origin[train_columns].copy()
df_test = df_test_origin[test_columns].copy()
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 1437.29 MB
Memory usage of optimization is 1507111539.00 MB
Decreased by -104857500.0%
Memory usage of dataframe is 227.78 MB
Memory usage of optimization is 238845728.00 MB
Decreased by -104857500.0%


,app_NEW_EXT_SOURCES_MEAN,app_AMT_CREDIT_PER_AMT_ANNUITY,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,app_DAYS_EMPLOYED_div_DAYS_BIRTH,DAYS_BIRTH,app_EXT_SOURCE_1_PER_DAYS_BIRTH,app_EXT_SOURCE_2_times_EXT_SOURCE_3,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_DEBT_MIN,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,CODE_GENDER,...,CARD_AMT_DRAWINGS_ATM_CURRENT_MAX,PREV_NAME_CONTRACT_STATUS_Unused_offer_MEAN,CLOSED_STATUS_C_MEAN_MEAN,CARD_CNT_INSTALMENT_MATURE_CUM_SUM,ACTIVE_AMT_ANNUITY_MAX,PREV_NAME_TYPE_SUITE_Children_MEAN,WEEKDAY_APPR_PROCESS_START_FRIDAY,REFUSED_NAME_CLIENT_TYPE_New_MEAN,SK_ID_CURR,TARGET
0,-2.046875,-0.596191,0.388672,-0.849609,1.597656,0.141479,-1.687500,-0.399170,-1.705078,-1.387695,...,-0.578125,-0.256836,-1.298828,-0.48877,-1.594727,-0.2771,-0.442383,0.768555,100002,1
1,-0.362793,1.686523,0.290283,-0.817871,-0.222778,-0.532715,1.546875,-0.328125,-0.032410,0.720215,...,-0.578125,-0.256836,0.801758,-0.48877,0.712891,-0.2771,-0.442383,0.768555,100003,0
2,0.889648,-0.109741,-1.044922,-1.407227,-0.716797,0.843262,0.272705,1.808594,-0.240479,-1.387695,...,-0.578125,-0.256836,0.801758,-0.48877,0.712891,-0.2771,-0.442383,0.768555,100004,0
3,0.950195,-1.558594,-0.514648,-0.159912,-0.708008,0.843262,1.546875,-0.403564,-0.032410,0.720215,...,1.771484,-0.256836,-1.491211,-0.48877,-1.594727,-0.2771,-0.442383,-1.336914,100006,0
4,-1.232422,0.326172,-1.044922,-0.206299,-0.901367,0.843262,1.546875,1.808594,0.416260,-1.387695,...,-0.578125,-0.256836,0.801758,-0.48877,0.712891,-0.2771,-0.442383,0.768555,100007,0


In [215]:
len(df_train.columns)

1226

In [216]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [217]:
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}
train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 1224) (61503, 1224)
[100]	training's auc: 0.810748	valid_1's auc: 0.777429
[200]	training's auc: 0.845697	valid_1's auc: 0.784888
[300]	training's auc: 0.870565	valid_1's auc: 0.785919
[400]	training's auc: 0.890372	valid_1's auc: 0.786908
[500]	training's auc: 0.906813	valid_1's auc: 0.786749
[auc] tr:0.889497, va0.7870
-------------------- 1 --------------------
(246009, 1224) (61502, 1224)
[100]	training's auc: 0.810568	valid_1's auc: 0.781296
[200]	training's auc: 0.844649	valid_1's auc: 0.789669
[300]	training's auc: 0.870434	valid_1's auc: 0.791096
[400]	training's auc: 0.891161	valid_1's auc: 0.791749
[500]	training's auc: 0.907747	valid_1's auc: 0.791641
[auc] tr:0.894289, va0.7919
-------------------- 2 --------------------
(246009, 1224) (61502, 1224)
[100]	training's auc: 0.810747	valid_1's auc: 0.782742
[200]	training's auc: 0.843741	valid_1's auc: 0.790008
[300]	training's auc: 0.869093	valid_1's auc: 0.791458
[400]	trai

In [218]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
1217,app_NEW_EXT_SOURCES_MEAN,112842.575104,820.388503
1193,app_AMT_CREDIT_PER_AMT_ANNUITY,7495.451350,260.296766
1203,app_DAYS_EMPLOYED_div_DAYS_BIRTH,4972.184438,622.286919
1194,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,4885.316122,386.679927
710,DAYS_BIRTH,4615.766086,545.162217
1212,app_EXT_SOURCE_2_times_EXT_SOURCE_3,4406.207184,360.878584
1206,app_EXT_SOURCE_1_PER_DAYS_BIRTH,4376.569661,341.876786
101,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_D...,3885.448201,1044.945211
780,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,3685.464831,365.599141
1192,app_AMT_CREDIT_AMT_GOODS_PRICE,3527.195984,554.742348


In [219]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [220]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [221]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_LgbFeatureSelection18.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.058468
1,100005,0.120277
2,100013,0.029495
3,100028,0.044094
4,100038,0.154820


### 変数選択019

In [222]:
len(df_train.columns)

1226

In [223]:
length = int(len(df_train.columns) * 0.95)
length

1164

In [224]:
cols = list(imp.sort_values("imp", ascending=False)['col'][:length])
if 'TARGET' not in cols:
    print('aa')
    
if 'SK_ID_CURR' not in cols:
    print('bb')
    
train_columns = cols.copy()
train_columns.append('SK_ID_CURR')
train_columns.append('TARGET')

test_columns = cols.copy()
test_columns.append('SK_ID_CURR')

aa
bb


In [225]:
del df_train
del df_test
gc.collect()
df_train = df_train_origin[train_columns].copy()
df_test = df_test_origin[test_columns].copy()
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 1366.91 MB
Memory usage of optimization is 1433308899.00 MB
Decreased by -104857500.0%
Memory usage of dataframe is 216.62 MB
Memory usage of optimization is 227147168.00 MB
Decreased by -104857500.0%


,app_NEW_EXT_SOURCES_MEAN,app_AMT_CREDIT_PER_AMT_ANNUITY,app_DAYS_EMPLOYED_div_DAYS_BIRTH,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,DAYS_BIRTH,app_EXT_SOURCE_2_times_EXT_SOURCE_3,app_EXT_SOURCE_1_PER_DAYS_BIRTH,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_DEBT_MIN,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,app_AMT_CREDIT_AMT_GOODS_PRICE,...,PREV_NAME_TYPE_SUITE_Other_B_MEAN,CLOSED_DPD_3_cnt_MIN_VAR,CLOSED_STATUS_C_MEAN_MEAN,FLOORSMAX_MODE,BURO_DPD_1_Month_MIN_MAX,REFUSED_SELLERPLACE_AREA_VAR,BURO_First_status_X_MEAN_VAR,BURO_Last_status_C_MEAN_MEAN,SK_ID_CURR,TARGET
0,-2.046875,-0.596191,-0.849609,0.388672,1.597656,-1.687500,0.141479,-0.399170,-1.705078,-0.032013,...,-0.209229,0.412354,-1.298828,-1.257812,-1.753906,0.542969,-0.995605,-0.560547,100002,1
1,-0.362793,1.686523,-0.817871,0.290283,-0.222778,1.546875,-0.532715,-0.328125,-0.032410,1.289062,...,-0.209229,0.412354,0.801758,-0.827148,0.563477,0.542969,0.806152,0.837402,100003,0
2,0.889648,-0.109741,-1.407227,-1.044922,-0.716797,0.272705,0.843262,1.808594,-0.240479,-0.790039,...,-0.209229,0.412354,0.801758,0.984375,0.563477,0.542969,0.806152,0.837402,100004,0
3,0.950195,-1.558594,-0.159912,-0.514648,-0.708008,1.546875,0.843262,-0.403564,-0.032410,-0.557129,...,-0.209229,-2.425781,-1.491211,0.984375,-1.784180,0.542969,-1.411133,-1.495117,100006,0
4,-1.232422,0.326172,-0.206299,-1.044922,-0.901367,1.546875,0.843262,1.808594,0.416260,-0.790039,...,-0.209229,0.412354,0.801758,0.984375,0.563477,0.542969,0.806152,0.837402,100007,0


In [226]:
len(df_train.columns)

1166

In [227]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [228]:
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 1164) (61503, 1164)
[100]	training's auc: 0.810992	valid_1's auc: 0.778614
[200]	training's auc: 0.845243	valid_1's auc: 0.786099
[300]	training's auc: 0.870109	valid_1's auc: 0.787285
[400]	training's auc: 0.889815	valid_1's auc: 0.7876
[500]	training's auc: 0.907345	valid_1's auc: 0.787814
[600]	training's auc: 0.921231	valid_1's auc: 0.788018
[700]	training's auc: 0.933055	valid_1's auc: 0.788294
[800]	training's auc: 0.942588	valid_1's auc: 0.788348
[900]	training's auc: 0.950571	valid_1's auc: 0.788335
[auc] tr:0.943175, va0.7885
-------------------- 1 --------------------
(246009, 1164) (61502, 1164)
[100]	training's auc: 0.810479	valid_1's auc: 0.780848
[200]	training's auc: 0.844422	valid_1's auc: 0.788856
[300]	training's auc: 0.868732	valid_1's auc: 0.790692
[400]	training's auc: 0.889605	valid_1's auc: 0.791403
[500]	training's auc: 0.906703	valid_1's auc: 0.791775
[auc] tr:0.904890, va0.7918
-------------------- 2 -------

In [229]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
1157,app_NEW_EXT_SOURCES_MEAN,112905.940924,1127.016729
1133,app_AMT_CREDIT_PER_AMT_ANNUITY,7691.127827,615.738996
1143,app_DAYS_EMPLOYED_div_DAYS_BIRTH,5057.563534,478.966013
672,DAYS_BIRTH,4950.760686,302.898949
1134,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,4774.596087,332.563817
1146,app_EXT_SOURCE_1_PER_DAYS_BIRTH,4521.047779,231.303251
1152,app_EXT_SOURCE_2_times_EXT_SOURCE_3,4335.881024,430.710991
96,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_D...,4064.179579,895.945672
741,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,3824.147513,216.159581
1132,app_AMT_CREDIT_AMT_GOODS_PRICE,3755.606243,387.627030


In [230]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [231]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [232]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_LgbFeatureSelection19.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.048043
1,100005,0.110114
2,100013,0.031256
3,100028,0.036632
4,100038,0.154070


### 変数選択020

In [233]:
len(df_train.columns)

1166

In [234]:
length = int(len(df_train.columns) * 0.95)
length

1107

In [235]:
cols = list(imp.sort_values("imp", ascending=False)['col'][:length])
if 'TARGET' not in cols:
    print('aa')
    
if 'SK_ID_CURR' not in cols:
    print('bb')
    
train_columns = cols.copy()
train_columns.append('SK_ID_CURR')
train_columns.append('TARGET')

test_columns = cols.copy()
test_columns.append('SK_ID_CURR')

aa
bb


In [236]:
del df_train
del df_test
gc.collect()
df_train = df_train_origin[train_columns].copy()
df_test = df_test_origin[test_columns].copy()
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 1300.05 MB
Memory usage of optimization is 1363196391.00 MB
Decreased by -104857500.0%
Memory usage of dataframe is 206.03 MB
Memory usage of optimization is 216033536.00 MB
Decreased by -104857500.0%


,app_NEW_EXT_SOURCES_MEAN,app_AMT_CREDIT_PER_AMT_ANNUITY,app_DAYS_EMPLOYED_div_DAYS_BIRTH,DAYS_BIRTH,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,app_EXT_SOURCE_1_PER_DAYS_BIRTH,app_EXT_SOURCE_2_times_EXT_SOURCE_3,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_DEBT_MIN,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,app_AMT_CREDIT_AMT_GOODS_PRICE,...,ACTIVE_DPD_1_Month_MIN_MAX,APPROVED_NAME_GOODS_CATEGORY_Clothing_and_Accessories_MEAN,APPROVED_NAME_CONTRACT_TYPE_Cash_loans_MEAN,AMT_REQ_CREDIT_BUREAU_MON,BURO_DPD_1_Month_MIN_SUM,ACTIVE_Month_closed_to_end_MIN_MIN,REFUSED_SELLERPLACE_AREA_VAR,CARD_card_AMT_DRAWINGS_CURRENT__AMT_DRAWINGS_OTHER_CURRENT_MEAN,SK_ID_CURR,TARGET
0,-2.046875,-0.596191,-0.849609,1.597656,0.388672,0.141479,-1.687500,-0.399170,-1.705078,-0.032013,...,-2.017578,-0.241821,-0.851562,-0.599609,3.273438,-1.426758,0.542969,-0.613770,100002,1
1,-0.362793,1.686523,-0.817871,-0.222778,0.290283,-0.532715,1.546875,-0.328125,-0.032410,1.289062,...,0.493408,-0.241821,0.847656,-0.599609,-0.325684,0.591797,0.542969,-0.613770,100003,0
2,0.889648,-0.109741,-1.407227,-0.716797,-1.044922,0.843262,0.272705,1.808594,-0.240479,-0.790039,...,0.493408,-0.241821,-0.851562,-0.599609,-0.325684,0.591797,0.542969,-0.613770,100004,0
3,0.950195,-1.558594,-0.159912,-0.708008,-0.514648,0.843262,1.546875,-0.403564,-0.032410,-0.557129,...,-2.031250,-0.241821,1.009766,2.009766,-0.325684,-1.779297,0.542969,1.735352,100006,0
4,-1.232422,0.326172,-0.206299,-0.901367,-1.044922,0.843262,1.546875,1.808594,0.416260,-0.790039,...,0.493408,-0.241821,1.417969,-0.599609,-0.325684,0.591797,0.542969,-0.613770,100007,0


In [237]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [238]:
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 1107) (61503, 1107)
[100]	training's auc: 0.810771	valid_1's auc: 0.777427
[200]	training's auc: 0.844792	valid_1's auc: 0.784294
[300]	training's auc: 0.870237	valid_1's auc: 0.78554
[400]	training's auc: 0.889728	valid_1's auc: 0.785698
[500]	training's auc: 0.907147	valid_1's auc: 0.785977
[600]	training's auc: 0.92104	valid_1's auc: 0.786309
[auc] tr:0.917419, va0.7864
-------------------- 1 --------------------
(246009, 1107) (61502, 1107)
[100]	training's auc: 0.810102	valid_1's auc: 0.781412
[200]	training's auc: 0.84525	valid_1's auc: 0.789413
[300]	training's auc: 0.869845	valid_1's auc: 0.790408
[400]	training's auc: 0.890254	valid_1's auc: 0.790689
[500]	training's auc: 0.906943	valid_1's auc: 0.791186
[600]	training's auc: 0.921926	valid_1's auc: 0.791276
[auc] tr:0.917456, va0.7913
-------------------- 2 --------------------
(246009, 1107) (61502, 1107)
[100]	training's auc: 0.810602	valid_1's auc: 0.782563
[200]	trainin

In [239]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
1100,app_NEW_EXT_SOURCES_MEAN,112771.561680,1047.237388
1076,app_AMT_CREDIT_PER_AMT_ANNUITY,7564.229065,752.768155
1086,app_DAYS_EMPLOYED_div_DAYS_BIRTH,5130.128359,504.331164
632,DAYS_BIRTH,5074.214322,246.661674
1077,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,4999.047601,312.319646
1089,app_EXT_SOURCE_1_PER_DAYS_BIRTH,4712.621716,125.702537
1095,app_EXT_SOURCE_2_times_EXT_SOURCE_3,4487.711628,217.421072
86,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_D...,4100.991156,1079.526333
701,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,3932.567430,387.845520
1075,app_AMT_CREDIT_AMT_GOODS_PRICE,3749.627934,480.533767


In [240]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [241]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [242]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_LgbFeatureSelection20.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.058012
1,100005,0.114733
2,100013,0.034330
3,100028,0.036448
4,100038,0.149179


### 変数選択021

In [243]:
len(df_train.columns)

1109

In [244]:
length = int(len(df_train.columns) * 0.95)
length

1053

In [245]:
cols = list(imp.sort_values("imp", ascending=False)['col'][:length])
if 'TARGET' not in cols:
    print('aa')
    
if 'SK_ID_CURR' not in cols:
    print('bb')
    
train_columns = cols.copy()
train_columns.append('SK_ID_CURR')
train_columns.append('TARGET')

test_columns = cols.copy()
test_columns.append('SK_ID_CURR')

aa
bb


In [246]:
del df_train
del df_test
gc.collect()
df_train = df_train_origin[train_columns].copy()
df_test = df_test_origin[test_columns].copy()
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 1236.70 MB
Memory usage of optimization is 1296774015.00 MB
Decreased by -104857500.0%
Memory usage of dataframe is 195.98 MB
Memory usage of optimization is 205504832.00 MB
Decreased by -104857500.0%


,app_NEW_EXT_SOURCES_MEAN,app_AMT_CREDIT_PER_AMT_ANNUITY,app_DAYS_EMPLOYED_div_DAYS_BIRTH,DAYS_BIRTH,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,app_EXT_SOURCE_1_PER_DAYS_BIRTH,app_EXT_SOURCE_2_times_EXT_SOURCE_3,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_DEBT_MIN,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,app_AMT_CREDIT_AMT_GOODS_PRICE,...,BURO_MONTHS_BALANCE_MIN_MIN,APPROVED_NAME_GOODS_CATEGORY_Clothing_and_Accessories_MEAN,BURO_AMT_ANNUITY_VAR,BURO_STATUS_X_MEAN_MEAN,REFUSED_HOUR_APPR_PROCESS_START_MAX,CLOSED_SK_ID_BUREAU_MIN_SUM,CLOSED_bureau_DAYS_CREDIT__CREDIT_DAY_OVERDUE_VAR,CARD_AMT_TOTAL_RECEIVABLE_MAX,SK_ID_CURR,TARGET
0,-2.046875,-0.596191,-0.849609,1.597656,0.388672,0.141479,-1.687500,-0.399170,-1.705078,-0.032013,...,-1.553711,-0.241821,-1.557617,-1.061523,0.758301,1.287109,-0.347656,-0.489014,100002,1
1,-0.362793,1.686523,-0.817871,-0.222778,0.290283,-0.532715,1.546875,-0.328125,-0.032410,1.289062,...,0.651367,-0.241821,0.719238,0.873535,0.758301,-0.766602,0.261475,-0.489014,100003,0
2,0.889648,-0.109741,-1.407227,-0.716797,-1.044922,0.843262,0.272705,1.808594,-0.240479,-0.790039,...,0.651367,-0.241821,0.719238,0.873535,0.758301,-0.766602,0.024277,-0.489014,100004,0
3,0.950195,-1.558594,-0.159912,-0.708008,-0.514648,0.843262,1.546875,-0.403564,-0.032410,-0.557129,...,0.651367,-0.241821,-1.557617,-1.285156,-0.980469,-0.766602,-1.752930,-0.489014,100006,0
4,-1.232422,0.326172,-0.206299,-0.901367,-1.044922,0.843262,1.546875,1.808594,0.416260,-0.790039,...,0.651367,-0.241821,0.719238,0.873535,0.758301,-0.766602,1.218750,-0.489014,100007,0


In [247]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [248]:
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 1053) (61503, 1053)
[100]	training's auc: 0.810504	valid_1's auc: 0.778514
[200]	training's auc: 0.845624	valid_1's auc: 0.786135
[300]	training's auc: 0.870906	valid_1's auc: 0.786601
[400]	training's auc: 0.890758	valid_1's auc: 0.786821
[auc] tr:0.885193, va0.7871
-------------------- 1 --------------------
(246009, 1053) (61502, 1053)
[100]	training's auc: 0.810241	valid_1's auc: 0.780785
[200]	training's auc: 0.844421	valid_1's auc: 0.789727
[300]	training's auc: 0.869464	valid_1's auc: 0.791347
[400]	training's auc: 0.889554	valid_1's auc: 0.792288
[500]	training's auc: 0.906164	valid_1's auc: 0.792038
[auc] tr:0.897588, va0.7923
-------------------- 2 --------------------
(246009, 1053) (61502, 1053)
[100]	training's auc: 0.810609	valid_1's auc: 0.782312
[200]	training's auc: 0.844141	valid_1's auc: 0.789172
[300]	training's auc: 0.86888	valid_1's auc: 0.790978
[400]	training's auc: 0.889306	valid_1's auc: 0.791863
[500]	train

In [249]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
1046,app_NEW_EXT_SOURCES_MEAN,112550.420831,1257.659889
1022,app_AMT_CREDIT_PER_AMT_ANNUITY,7296.487948,562.283262
1023,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,4872.934829,197.667230
1032,app_DAYS_EMPLOYED_div_DAYS_BIRTH,4847.433805,460.923476
602,DAYS_BIRTH,4679.309706,318.581676
1035,app_EXT_SOURCE_1_PER_DAYS_BIRTH,4386.032012,223.842706
1041,app_EXT_SOURCE_2_times_EXT_SOURCE_3,4219.788895,314.862023
80,ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_D...,3940.742475,948.341072
670,INS_ins_DAYS_ENTRY_PAYMENT_DAYS_INSTALMENT_MAX,3595.575191,225.586561
599,CODE_GENDER,3477.583220,265.667011


In [250]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [251]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [252]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_LgbFeatureSelection21.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.050896
1,100005,0.109332
2,100013,0.032331
3,100028,0.037270
4,100038,0.155683


### 変数選択022

### 変数選択023

### 変数選択024

### 変数選択025

### 変数選択026

### 変数選択027

### 変数選択028

### 変数選択029

### 変数選択030

### 変数選択031

### 変数選択032

### 変数選択033

### 変数選択034

### 変数選択035

In [253]:
baseline = Baseline('SK_ID_CURR')

sampler = optuna.samplers.TPESampler(seed=123)
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(objective, n_trials=50, n_jobs=5)

[I 2022-06-27 08:28:11,991] A new study created in memory with name: no-name-4b5e37df-b93d-4f68-b865-9223d1ff284a
[I 2022-06-27 08:34:53,045] Trial 0 finished with value: 0.7870544624212851 and parameters: {'num_leaves': 69, 'min_child_samples': 131, 'min_sum_hessian_in_leaf': 0.0029386882627703125, 'feature_fraction': 0.9512156486284564, 'bagging_fraction': 0.5071733172647104, 'lambda_l1': 11.794853171333024, 'lambda_l2': 0.01812867963167791}. Best is trial 0 with value: 0.7870544624212851.
[I 2022-06-27 08:34:53,269] Trial 4 finished with value: 0.7870544624212851 and parameters: {'num_leaves': 220, 'min_child_samples': 114, 'min_sum_hessian_in_leaf': 5.2564008946518467e-05, 'feature_fraction': 0.882679616887935, 'bagging_fraction': 0.7817479308321091, 'lambda_l1': 67.63583517713327, 'lambda_l2': 19.738992272834807}. Best is trial 0 with value: 0.7870544624212851.
[I 2022-06-27 08:34:53,953] Trial 2 finished with value: 0.7870544624212851 and parameters: {'num_leaves': 86, 'min_child

[I 2022-06-27 09:02:53,496] Trial 24 finished with value: 0.7870544624212851 and parameters: {'num_leaves': 194, 'min_child_samples': 188, 'min_sum_hessian_in_leaf': 0.0007827299992072825, 'feature_fraction': 0.7071712102170387, 'bagging_fraction': 0.5119953597448146, 'lambda_l1': 24.009268158449423, 'lambda_l2': 0.06983091971904934}. Best is trial 0 with value: 0.7870544624212851.
[I 2022-06-27 09:02:55,446] Trial 21 finished with value: 0.7870544624212851 and parameters: {'num_leaves': 179, 'min_child_samples': 139, 'min_sum_hessian_in_leaf': 1.372055510239543e-05, 'feature_fraction': 0.9423709360932294, 'bagging_fraction': 0.5089576586340028, 'lambda_l1': 61.90875939396014, 'lambda_l2': 0.07986745551165396}. Best is trial 0 with value: 0.7870544624212851.
[I 2022-06-27 09:02:57,066] Trial 22 finished with value: 0.7870544624212851 and parameters: {'num_leaves': 169, 'min_child_samples': 142, 'min_sum_hessian_in_leaf': 0.0008149367390501666, 'feature_fraction': 0.7123312105089946, 'b

[I 2022-06-27 09:33:26,607] Trial 43 finished with value: 0.7870544624212851 and parameters: {'num_leaves': 22, 'min_child_samples': 143, 'min_sum_hessian_in_leaf': 0.00015168898478091247, 'feature_fraction': 0.7632422395718983, 'bagging_fraction': 0.517032940478842, 'lambda_l1': 0.4152262424879434, 'lambda_l2': 0.46927048268505644}. Best is trial 0 with value: 0.7870544624212851.
[I 2022-06-27 09:33:28,044] Trial 42 finished with value: 0.7870544624212851 and parameters: {'num_leaves': 23, 'min_child_samples': 135, 'min_sum_hessian_in_leaf': 0.0001592490493234275, 'feature_fraction': 0.7625967873039254, 'bagging_fraction': 0.5046400898582202, 'lambda_l1': 93.10873371264582, 'lambda_l2': 0.43160690821343517}. Best is trial 0 with value: 0.7870544624212851.
[I 2022-06-27 09:33:29,708] Trial 44 finished with value: 0.7870544624212851 and parameters: {'num_leaves': 39, 'min_child_samples': 111, 'min_sum_hessian_in_leaf': 0.0010991637012374774, 'feature_fraction': 0.7164380505863198, 'bagg

In [254]:
trial = study.best_trial
print("auc(best)={:.4f}".format(trial.value))
display(trial.params)

auc(best)=0.7871


{'num_leaves': 69,
 'min_child_samples': 131,
 'min_sum_hessian_in_leaf': 0.0029386882627703125,
 'feature_fraction': 0.9512156486284564,
 'bagging_fraction': 0.5071733172647104,
 'lambda_l1': 11.794853171333024,
 'lambda_l2': 0.01812867963167791}

In [255]:
params_best = trial.params
params_best.update(params_base)
display(params_best)

{'num_leaves': 69,
 'min_child_samples': 131,
 'min_sum_hessian_in_leaf': 0.0029386882627703125,
 'feature_fraction': 0.9512156486284564,
 'bagging_fraction': 0.5071733172647104,
 'lambda_l1': 11.794853171333024,
 'lambda_l2': 0.01812867963167791,
 'boosting_type': 'gbdt',
 'objective': 'binary',
 'metric': 'auc',
 'verbosity': -1,
 'learning_rate': 0.05,
 'n_estimators': 100000,
 'random_state': 123,
 'bagging_freq': 1}

In [256]:
train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5
                                    ,params=params_best)

-------------------- 0 --------------------
(246008, 1053) (61503, 1053)
[LightGBM] [Warning] feature_fraction is set=0.9512156486284564, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9512156486284564
[LightGBM] [Warning] lambda_l1 is set=11.794853171333024, reg_alpha=0.0 will be ignored. Current value: lambda_l1=11.794853171333024
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.0029386882627703125, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.0029386882627703125
[LightGBM] [Warning] bagging_fraction is set=0.5071733172647104, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5071733172647104
[LightGBM] [Warning] lambda_l2 is set=0.01812867963167791, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.01812867963167791
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[100]	training's auc: 0.821951	valid_1's auc: 0.779774
[200]	training's a

In [257]:
# 推論用データセットの作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

# predict
test_pred = baseline.predict_lgb(x_test
                       ,id_test
                       ,list_nfold=[0,1,2,3,4])

# make submission-file
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv("./home-credit-default-risk/submit/submission_LgbFeatureSelection39.csv", index=None)

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.
(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.052232
1,100005,0.080591
2,100013,0.044494
3,100028,0.045518
4,100038,0.109767


### 変数選択040

In [258]:
length = 1053

In [259]:
cols = list(imp.sort_values("imp", ascending=False)['col'][:length])
if 'TARGET' not in cols:
    print('aa')
    
if 'SK_ID_CURR' not in cols:
    print('bb')
    
train_columns = cols.copy()
train_columns.append('SK_ID_CURR')
train_columns.append('TARGET')

test_columns = cols.copy()
test_columns.append('SK_ID_CURR')

aa
bb


In [260]:
del df_train
del df_test
gc.collect()
df_train = df_train_origin[train_columns].copy()
df_test = df_test_origin[test_columns].copy()
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 1236.70 MB
Memory usage of optimization is 1296774015.00 MB
Decreased by -104857500.0%
Memory usage of dataframe is 195.98 MB
Memory usage of optimization is 205504832.00 MB
Decreased by -104857500.0%


,app_NEW_EXT_SOURCES_MEAN,DAYS_BIRTH,DAYS_ID_PUBLISH,app_AMT_CREDIT_PER_AMT_ANNUITY,AMT_ANNUITY,DAYS_REGISTRATION,app_DAYS_EMPLOYED_div_DAYS_BIRTH,REGION_POPULATION_RELATIVE,EXT_SOURCE_2,app_EXT_SOURCE_1_PER_DAYS_BIRTH,...,CARD_AMT_DRAWINGS_CURRENT_MIN,ORGANIZATION_TYPE_Industry__type_9,OCCUPATION_TYPE_Low_skill_Laborers,REFUSED_CODE_REJECT_REASON_SCOFR_MEAN,FLAG_DOCUMENT_18,ORGANIZATION_TYPE_Realtor,NAME_HOUSING_TYPE_Office_apartment,ACTIVE_CREDIT_TYPE_Microloan_MEAN,SK_ID_CURR,TARGET
0,-2.046875,1.597656,0.553711,-0.596191,0.028198,0.171143,-0.849609,0.046997,-1.306641,0.141479,...,-0.035004,-0.105225,-0.082764,0.775879,-0.090515,-0.035919,-0.092651,-0.430420,100002,1
1,-0.362793,-0.222778,1.837891,1.686523,0.744141,1.122070,-0.817871,-1.730469,0.548828,-0.532715,...,-0.035004,-0.105225,-0.082764,0.775879,-0.090515,-0.035919,-0.092651,-0.430420,100003,0
2,0.889648,-0.716797,0.281250,-0.109741,-2.085938,-0.007088,-1.407227,-0.863281,0.207520,0.843262,...,-0.035004,-0.105225,-0.082764,0.775879,-0.090515,-0.035919,-0.092651,2.335938,100004,0
3,0.950195,-0.708008,0.343262,-1.558594,0.378662,-1.244141,-0.159912,-1.112305,0.695312,0.843262,...,-0.035004,-0.105225,-0.082764,-1.311523,-0.090515,-0.035919,-0.092651,-0.430420,100006,0
4,-1.232422,-0.901367,-0.322754,0.326172,-0.196533,-0.021057,-0.206299,0.799805,-0.998047,0.843262,...,-0.035004,-0.105225,-0.082764,0.775879,-0.090515,-0.035919,-0.092651,2.335938,100007,0


In [261]:
### infをnanへ置換
df_train = df_train.replace([np.inf, -np.inf], np.nan)
df_train.head()

,app_NEW_EXT_SOURCES_MEAN,DAYS_BIRTH,DAYS_ID_PUBLISH,app_AMT_CREDIT_PER_AMT_ANNUITY,AMT_ANNUITY,DAYS_REGISTRATION,app_DAYS_EMPLOYED_div_DAYS_BIRTH,REGION_POPULATION_RELATIVE,EXT_SOURCE_2,app_EXT_SOURCE_1_PER_DAYS_BIRTH,...,CARD_AMT_DRAWINGS_CURRENT_MIN,ORGANIZATION_TYPE_Industry__type_9,OCCUPATION_TYPE_Low_skill_Laborers,REFUSED_CODE_REJECT_REASON_SCOFR_MEAN,FLAG_DOCUMENT_18,ORGANIZATION_TYPE_Realtor,NAME_HOUSING_TYPE_Office_apartment,ACTIVE_CREDIT_TYPE_Microloan_MEAN,SK_ID_CURR,TARGET
0,-2.046875,1.597656,0.553711,-0.596191,0.028198,0.171143,-0.849609,0.046997,-1.306641,0.141479,...,-0.035004,-0.105225,-0.082764,0.775879,-0.090515,-0.035919,-0.092651,-0.430420,100002,1
1,-0.362793,-0.222778,1.837891,1.686523,0.744141,1.122070,-0.817871,-1.730469,0.548828,-0.532715,...,-0.035004,-0.105225,-0.082764,0.775879,-0.090515,-0.035919,-0.092651,-0.430420,100003,0
2,0.889648,-0.716797,0.281250,-0.109741,-2.085938,-0.007088,-1.407227,-0.863281,0.207520,0.843262,...,-0.035004,-0.105225,-0.082764,0.775879,-0.090515,-0.035919,-0.092651,2.335938,100004,0
3,0.950195,-0.708008,0.343262,-1.558594,0.378662,-1.244141,-0.159912,-1.112305,0.695312,0.843262,...,-0.035004,-0.105225,-0.082764,-1.311523,-0.090515,-0.035919,-0.092651,-0.430420,100006,0
4,-1.232422,-0.901367,-0.322754,0.326172,-0.196533,-0.021057,-0.206299,0.799805,-0.998047,0.843262,...,-0.035004,-0.105225,-0.082764,0.775879,-0.090515,-0.035919,-0.092651,2.335938,100007,0


In [262]:
### 欠損値補完
df_train = df_train.fillna(df_train.max() + (df_train.max() * 0.4))
df_train.head()

,app_NEW_EXT_SOURCES_MEAN,DAYS_BIRTH,DAYS_ID_PUBLISH,app_AMT_CREDIT_PER_AMT_ANNUITY,AMT_ANNUITY,DAYS_REGISTRATION,app_DAYS_EMPLOYED_div_DAYS_BIRTH,REGION_POPULATION_RELATIVE,EXT_SOURCE_2,app_EXT_SOURCE_1_PER_DAYS_BIRTH,...,CARD_AMT_DRAWINGS_CURRENT_MIN,ORGANIZATION_TYPE_Industry__type_9,OCCUPATION_TYPE_Low_skill_Laborers,REFUSED_CODE_REJECT_REASON_SCOFR_MEAN,FLAG_DOCUMENT_18,ORGANIZATION_TYPE_Realtor,NAME_HOUSING_TYPE_Office_apartment,ACTIVE_CREDIT_TYPE_Microloan_MEAN,SK_ID_CURR,TARGET
0,-2.046875,1.597656,0.553711,-0.596191,0.028198,0.171143,-0.849609,0.046997,-1.306641,0.141479,...,-0.035004,-0.105225,-0.082764,0.775879,-0.090515,-0.035919,-0.092651,-0.430420,100002,1
1,-0.362793,-0.222778,1.837891,1.686523,0.744141,1.122070,-0.817871,-1.730469,0.548828,-0.532715,...,-0.035004,-0.105225,-0.082764,0.775879,-0.090515,-0.035919,-0.092651,-0.430420,100003,0
2,0.889648,-0.716797,0.281250,-0.109741,-2.085938,-0.007088,-1.407227,-0.863281,0.207520,0.843262,...,-0.035004,-0.105225,-0.082764,0.775879,-0.090515,-0.035919,-0.092651,2.335938,100004,0
3,0.950195,-0.708008,0.343262,-1.558594,0.378662,-1.244141,-0.159912,-1.112305,0.695312,0.843262,...,-0.035004,-0.105225,-0.082764,-1.311523,-0.090515,-0.035919,-0.092651,-0.430420,100006,0
4,-1.232422,-0.901367,-0.322754,0.326172,-0.196533,-0.021057,-0.206299,0.799805,-0.998047,0.843262,...,-0.035004,-0.105225,-0.082764,0.775879,-0.090515,-0.035919,-0.092651,2.335938,100007,0


In [263]:
### infをnanへ置換
df_test = df_test.replace([np.inf, -np.inf], np.nan)
df_test.head()

,app_NEW_EXT_SOURCES_MEAN,DAYS_BIRTH,DAYS_ID_PUBLISH,app_AMT_CREDIT_PER_AMT_ANNUITY,AMT_ANNUITY,DAYS_REGISTRATION,app_DAYS_EMPLOYED_div_DAYS_BIRTH,REGION_POPULATION_RELATIVE,EXT_SOURCE_2,app_EXT_SOURCE_1_PER_DAYS_BIRTH,...,ACTIVE_AMT_CREDIT_SUM_OVERDUE_MAX,CARD_AMT_DRAWINGS_CURRENT_MIN,ORGANIZATION_TYPE_Industry__type_9,OCCUPATION_TYPE_Low_skill_Laborers,REFUSED_CODE_REJECT_REASON_SCOFR_MEAN,FLAG_DOCUMENT_18,ORGANIZATION_TYPE_Realtor,NAME_HOUSING_TYPE_Office_apartment,ACTIVE_CREDIT_TYPE_Microloan_MEAN,SK_ID_CURR
0,0.327393,-0.757324,1.453125,0.810059,-0.307861,-0.249634,-0.418945,0.050568,1.415039,-1.323242,...,-0.437256,-0.035004,-0.105225,-0.082764,0.775879,-0.090515,-0.035919,-0.092651,-0.430420,100001
1,-0.598145,-0.507324,0.888672,-1.159180,-0.604980,-1.111328,0.302002,1.211914,-1.159180,-1.096680,...,-0.437256,-0.035004,-0.105225,12.078125,0.775879,-0.090515,-0.035919,-0.092651,-0.430420,100005
2,0.988281,-0.923340,-0.351562,-1.750000,2.203125,0.674805,0.185181,0.073669,0.950195,0.865723,...,2.306641,-0.035004,-0.105225,-0.082764,0.775879,-0.090515,-0.035919,-0.092651,2.335938,100013
3,0.197998,0.421875,-0.802246,1.282227,1.408203,0.745117,-0.331543,0.648438,-0.030289,-1.285156,...,-0.437256,-0.035004,-0.105225,-0.082764,0.775879,-0.090515,-0.035919,-0.092651,-0.430420,100028
4,-1.281250,0.650391,-0.836914,-0.175781,0.529785,0.066895,-0.110596,-0.863281,-0.465576,-0.350830,...,-0.437256,-0.035004,-0.105225,-0.082764,0.775879,-0.090515,-0.035919,-0.092651,-0.430420,100038


In [264]:
### 欠損値補完
df_test = df_test.fillna(df_test.max() + (df_test.max() * 0.4))
df_test.head()

,app_NEW_EXT_SOURCES_MEAN,DAYS_BIRTH,DAYS_ID_PUBLISH,app_AMT_CREDIT_PER_AMT_ANNUITY,AMT_ANNUITY,DAYS_REGISTRATION,app_DAYS_EMPLOYED_div_DAYS_BIRTH,REGION_POPULATION_RELATIVE,EXT_SOURCE_2,app_EXT_SOURCE_1_PER_DAYS_BIRTH,...,ACTIVE_AMT_CREDIT_SUM_OVERDUE_MAX,CARD_AMT_DRAWINGS_CURRENT_MIN,ORGANIZATION_TYPE_Industry__type_9,OCCUPATION_TYPE_Low_skill_Laborers,REFUSED_CODE_REJECT_REASON_SCOFR_MEAN,FLAG_DOCUMENT_18,ORGANIZATION_TYPE_Realtor,NAME_HOUSING_TYPE_Office_apartment,ACTIVE_CREDIT_TYPE_Microloan_MEAN,SK_ID_CURR
0,0.327393,-0.757324,1.453125,0.810059,-0.307861,-0.249634,-0.418945,0.050568,1.415039,-1.323242,...,-0.437256,-0.035004,-0.105225,-0.082764,0.775879,-0.090515,-0.035919,-0.092651,-0.430420,100001
1,-0.598145,-0.507324,0.888672,-1.159180,-0.604980,-1.111328,0.302002,1.211914,-1.159180,-1.096680,...,-0.437256,-0.035004,-0.105225,12.078125,0.775879,-0.090515,-0.035919,-0.092651,-0.430420,100005
2,0.988281,-0.923340,-0.351562,-1.750000,2.203125,0.674805,0.185181,0.073669,0.950195,0.865723,...,2.306641,-0.035004,-0.105225,-0.082764,0.775879,-0.090515,-0.035919,-0.092651,2.335938,100013
3,0.197998,0.421875,-0.802246,1.282227,1.408203,0.745117,-0.331543,0.648438,-0.030289,-1.285156,...,-0.437256,-0.035004,-0.105225,-0.082764,0.775879,-0.090515,-0.035919,-0.092651,-0.430420,100028
4,-1.281250,0.650391,-0.836914,-0.175781,0.529785,0.066895,-0.110596,-0.863281,-0.465576,-0.350830,...,-0.437256,-0.035004,-0.105225,-0.082764,0.775879,-0.090515,-0.035919,-0.092651,-0.430420,100038


In [265]:
### 処理対象の列名を配列化
encoding_columns = list(df_test.select_dtypes(include='number').columns)
encoding_columns.remove('SK_ID_CURR')

In [1]:
### Yao-Jhonson変換
#for column in encoding_columns:
#    modules.process_yj_convert(df_train, df_test, column, [column])

#df_train.head()

In [ ]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

In [ ]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

In [ ]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

In [ ]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [ ]:


params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

In [ ]:
imp.sort_values("imp", ascending=False)[:20]

In [ ]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [ ]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

In [ ]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_LgbFeatureSelection40.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

### 変数選択041

In [ ]:
baseline = Baseline('SK_ID_CURR')

sampler = optuna.samplers.TPESampler(seed=123)
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(objective, n_trials=50, n_jobs=5)

In [ ]:
trial = study.best_trial
print("auc(best)={:.4f}".format(trial.value))
display(trial.params)

In [ ]:
params_best = trial.params
params_best.update(params_base)
display(params_best)

In [ ]:
train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5
                                    ,params=params_best)

In [ ]:
# 推論用データセットの作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

# predict
test_pred = baseline.predict_lgb(x_test
                       ,id_test
                       ,list_nfold=[0,1,2,3,4])

# make submission-file
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv("./home-credit-default-risk/submit/submission_LgbFeatureSelection41.csv", index=None)

In [ ]:
del df_train
del df_train_origin
del df_test
del df_test_origin
gc.collect()